# Diagramas de Voronoi – como vota meu vizinho?
Esse script gera os [diagramas de Voronoi](https://pt.wikipedia.org/wiki/Diagrama_de_Voronoy) para a matéria sobre a geografia detalhada no voto nas Eleições de 2018. Ele cria o diagrama com base nos votos **presidenciais**. Com os devidos ajustes nas bases de dados, pode gerar polígonos para qualquer outro cargo. O usuário apenas precisa ficar atento para evitar que cargos com muitos candidatos (especialmente deputados estaduais e federais) superem os limites de colunas dos arquivos `.shp`.

Note que, nesse caso, estamos usando os dados do arquivo de **boletim de urna** do TSE.

### 1. Preparação dos dados e pacotes necessários para análise

Importação de bibliotecas.

In [1]:
# Precisamos de muuuuuitas coisas
import pandas as pd
import geopandas as gpd
import geopandas as gpd
import numpy as np
import shapely
from shapely.geometry import Point, LineString, Polygon, MultiPolygon
import requests
import time
import googlemaps
import glob
import scipy.spatial as spatial
from scipy.spatial import Voronoi, voronoi_plot_2d
import matplotlib.pyplot as plt
import matplotlib.path as path
import matplotlib as mpl
from functools import reduce
from datetime import datetime
import re
import os
import shutil
import sys
import linecache
from tqdm._tqdm_notebook import tqdm_notebook as tqdm # Progress monitoring
import fiona

# Define configurações
tqdm.pandas()
%matplotlib inline
pd.options.display.max_rows = 100

Preparação da chave da API.

In [2]:
# API Key do Google Maps
gmaps = googlemaps.Client(key='INSIRA SUA CHAVE DE API AQUI') 

Lê arquivo com lista de locais de votação de 2018 e deixa ele no formato necessário.

In [3]:
dtype = {'SGL_UF':'str', 'COD_LOCALIDADE_IBGE':'str', 'LOCALIDADE_LOCAL_VOTACAO':'str', 'ZONA':'str',
    'BAIRRO_ZONA_SEDE':'str', 'LATITUDE_ZONA':'float', 'LONGITUDE_ZONA':'float',
    'NUM_LOCAL':'str', 'SITUACAO_LOCAL':'str', 'TIPO_LOCAL':'str', 'LOCAL_VOTACAO':'str', 
    'ENDERECO':'str', 'BAIRRO_LOCAL_VOT':'str', 'CEP':'str', 'LATITUDE_LOCAL':'float', 
    'LONGITUDE_LOCAL':'float', 'NUM_SECAO':'str', 'SECAO_AGREGADORA':'str', 'SECAO_AGREGADA':'str'
    }
base = pd.read_csv('../data/locais/local-votacao-08-08-2018.csv', encoding='Latin5', sep=';',decimal=',', dtype=dtype)

O próximo passo é tirar a sujeira de parte desse banco de dados para facilitar a geolocalização. Depois de alguma manual, percebemos alguns padrões que podem ser evitados. Os locais de Salvador, por exemplo, são identificados como "Zona Urbana" e "Zona Rural". Isso é desnecessário e complica o trabalho do Google na hora de localizar. Vamos remover.

In [4]:
base['ENDERECO'] = base.ENDERECO.str.replace(" - ZONA URBANA","")
base['ENDERECO'] = base.ENDERECO.str.replace(" - ZONA RURAL","")

In [5]:
# Cria um campo único para evitar consultas duplas
base['local_unico'] = base.LOCAL_VOTACAO + ', ' + base.ENDERECO + ', ' + base.LOCALIDADE_LOCAL_VOTACAO + ', ' + base.SGL_UF

In [6]:
# Cria um id único com base na query - permite juntar cada par de coordenadas a uma localidade única
base['id_unico'] = base.groupby('local_unico').grouper.group_info[0].astype(str)
base.to_csv("../data/locais/base-por-secao.csv", index=False)

In [7]:
# Cria uma cópia do campo, sem duplicatas - útil para geolocalizar cada endereço apenas uma vez
locais = base.drop_duplicates(subset='id_unico').reset_index()

In [8]:
# Transforma o campo da variável locais em string.
# Por algum motivo, ao fazer o subset, o dado voltou a ser tratado com int.
locais.id_unico = locais.id_unico.astype(str)

In [9]:
# Adiciona um campo para o país
locais['COUNTRY'] = 'BRASIL'

Preenche campos com valores temporários, mas que serão necessários para iniciar o processo de geolocalização.

In [10]:
def get_precision(row):

    # Função simples para determinar se a escola já tem um valor de lat/long
    # Se tiver, isso significa que ela já foi geocodificada pelo TSE
    # Assim, definimos uma categoria de precisão: "TSE"
    # As restantes serão classificadas, temporariamente, como None
    # E terão esse valor preenchido futuramente

    
    if np.isnan(row.LATITUDE_LOCAL):
        precision = 'NO_VALUE'
        fetched_address = 'NO_VALUE'
        lon = np.nan
        lat = np.nan
        
    else:
        precision = 'TSE'
        fetched_address = 'TSE'
        lon = row.LONGITUDE_LOCAL
        lat = row.LATITUDE_LOCAL
        
    return pd.Series({
            "precision":precision,
            "fetched_address":fetched_address,
            "lat":lat,
            "lon":lon,
            })

locais[["fetched_address", "lat", "lon", "precision"]] = locais.apply(get_precision, axis=1)

Deleta campos que se tornaram redundantes depois dessa preparação. Também reordena as colunas de modo mais intuitivo.

In [11]:
locais = locais[['id_unico', 'local_unico', 'lat', 'lon','fetched_address','precision',
                 'LOCAL_VOTACAO', 'ENDERECO', 'CEP',
                'COD_LOCALIDADE_IBGE', 'LOCALIDADE_LOCAL_VOTACAO', 'SGL_UF', 
                'ZONA', 'BAIRRO_ZONA_SEDE', 'LATITUDE_ZONA', 'LONGITUDE_ZONA',
                'NUM_SECAO', 'NUM_LOCAL','BAIRRO_LOCAL_VOT', 'COUNTRY']]

Essa é a estrutura de dados com a qual iremos trabalhar.

### 2. Geolocalização

Usando a API do Google, geoferenciamos os locais de votação das Eleições de 2018 nas oito principais capitais do Brasil.

**Input:** um arquivo produzido pelo TSE com a localização das **sessões eleitorais**. Alguns pontos já tem latitude e longitude, mas nem todos. Eles estão organizados por sessão – ou seja, por urna. Ele foi tratado previamente no Open Refine para padronizar alguns campos, como nomes de rua com erros de digitação.
    
**Output:** um arquivo com **locais de votação** únicos, com latitude e longitude para todos os itens.

Para isso, vamos definir uma função para geolocalizar os elementos necessários.

In [12]:
# Função para localizar os colégios eleitorais que não foram georeferenciados pelo TSE
def get_coordinates(row, query_params, exception_list):
    ### Essa função deve ser rodada pelo método df.apply() do Pandas
    ### Ela recebe um parâmetro 'query', que se refere ao tipo de busca que queremos realizar.   
    ### Ele deve ser passado na forma de uma lista com a combinação das colunas que vão ser passadas para a API, em ordem.
    ### Considere, por exemplo, query = ['ENDERECO', 'LOCALIDADE_LOCAL_VOTACAO', 'SGL_UF'].
    ### Isso faria uma busca por "Rua A, Cidade B, DF" no Google Maps.
    ### Note que ele checa se a precisão da linha é superior a precisão encontrada na request antes de substituir.
    ### exception_list é necessariamente uma lista vazia, que serve para guardar as exceções item a item.
    
    ##### INÍCIO DAS FUNCIONALIDADES ####
    
    # Aqui, atribuímos um valor numérico para cada uma das possíveis classificações de precisão.
    # Assim, podemos checar se a precisão que recuperamos ao enviar a requisição é melhor que a atual.
    # Se for assim, atualizamos. Caso contrário, mantemos o anterior.
    precision_order = {
                       'TSE':1,
                       'ROOFTOP':2,
                       'RANGE_INTERPOLATED':3,
                       'GEOMETRIC_CENTER':4,
                       'APPROXIMATE':5,
                       'NO_VALUE':6,
                       'ERROR':7,
                      } 
    
    # Agora, precisamos construir a query customizada que foi passada como parâmetro
    query = [str(row[item]).strip() for item in query_params]
    query = ', '.join(query)

    # Com esses parâmetros definidos, já podemos começar a georeferenciar    
    try:
        # Primeiro, verificamos se o item que queremos avaliar já tem uma precisão adequada.
        # Ou seja, se tem uma precisão de nível 1 ou 2 ('TSE' e 'ROOFTOP').
        # Caso positivo, podemos ignorar e manter os valores antigos.
        # Caso contrário, segue para o referenciamento.
        if precision_order[row.precision] < 3:
            lat = row.lat
            lon = row.lon
            precision = row.precision
            fetched_address = row.fetched_address
        
        # Aqui, já sabemos que o item não alcança nosso nível mínimo de precisão.
        else:
            
            # Envia a requisição para o Google.
            geocode_result = gmaps.geocode(query)

            # Descobre a precisão do resultado obtido.
            precision = geocode_result[0]['geometry']['location_type']

            # Se a precisão for maior do que a observada, podemos substituir.
            # Essa checagem é importante para evitar que um resultado com precisão 'RANGE_INTERPOLATED'
            # seja trocado por um 'ERROR', por exemplo.
            if precision_order[precision] < precision_order[row.precision]:

                lat = geocode_result[0]['geometry']['location']['lat']
                lon = geocode_result[0]['geometry']['location']['lng']
                fetched_address = geocode_result[0]['formatted_address']

            # Se não, mantemos o valor antigo.
            else:
                lat = row.lat
                lon = row.lon
                precision = row.precision
                fetched_address = row.fetched_address

    # Caso o georeferenciamento falhe, mantemos o valor que existia anteriormente.
    except:
        print('ERROR!')
        print()
        exception_list.append(row.local_unico)
        lat = row.lat
        lon = row.lon
        fetched_address = row.fetched_address
        precision = row.precision
            
    return pd.Series({
        'lon':lon,
        'lat':lat,
        'precision':precision,
        'fetched_address':fetched_address,
    })

Primeiro, vamos rodar a função cidade por cidade, para evitar que um desconexão cause a perda de todo o progresso. Os arquivos são salvos sempre que o geocoding de uma cidade acabar.

In [13]:
def geocode_each_city(cities, df, query_params, try_no, exception_list):
    
    ### Essa função roda o código da função get_coordinates(row, query_params, exception_list)
    ### A diferença é que faz isso aos poucos, cidade por cidade, salvando um arquivo sempre que termina.
    ### Os argumentos são os seguintes:
    ### cities: lista de cidades para geolocalizar
    ### df: o dataframe com os locais de votação
    ### query_paramd: a lista de campos que compõe a request que deve ser enviada para o goOGLE
    ### try_no: uma string que vai servir como identificador dos arquivos. Ela, idealmente,
    ### deve ser um número. Assim, conseguimos controlar versões depois de rodar o código diversas vezes
    ### com mudanças na variável query_param.
    ### exception_list: a mesma lista de exceções da função get_coordinates(...)
    
    for city in cities:

        # Cria um df filtrado coma  cidade que queremos
        temp = df[df.COD_LOCALIDADE_IBGE==city]
        
        # Salva os dados para o df filtrado
        data = temp.apply(get_coordinates, args=(query_params,exception_list), axis=1)
        
        # Passa os dados para o df original
        temp[data.columns] = data
        
        # Salva uma primeira versão para não precisar repetir todo o processo
        temp.to_csv('../data/geocode/geocode-' + str(try_no) + '-' + 'try' + str(city) + '.csv', index=False)
        
        # Também salva um log de excessões
        newfile = '../data/error-logs/errorlog'+ str(try_no) + '-' + 'try' + str(city) + '.txt'
        exception_dump = '\n'.join(exception_list)
        
        with open(newfile, 'w') as outfile:
            outfile.write(exception_dump)

Como temos vários arquivos salvos depois do processo de geolocalização, precisamos concatená-los em um único arquivo depois de terminar o processo.

In [14]:
def concatenate_files(try_no):
    
    ### Essa função lê os arquivos que foram salvos ao executar geocode_each_city(cities, df, query_params, try_no).
    ### Depois, concatena todos os dataframes e retorna uma única variável.
    ### O único argumento é try_no – o mesmo da função geocode.
    ### Sua função é identificar quais arquivos devem ser concantenados.
    
    files = glob.glob("../data/geocode/geocode-" + str(try_no) + "*.csv")
    dfs = []
    dtype = {
        'SGL_UF':'str', 'COD_LOCALIDADE_IBGE':'str', 'LOCALIDADE_LOCAL_VOTACAO':'str', 'ZONA':'str',
        'BAIRRO_ZONA_SEDE':'str', 'LATITUDE_ZONA':'float', 'LONGITUDE_ZONA':'float',
        'NUM_LOCAL':'str', 'SITUACAO_LOCAL':'str', 'TIPO_LOCAL':'str', 'LOCAL_VOTACAO':'str', 
        'ENDERECO':'str', 'BAIRRO_LOCAL_VOT':'str', 'CEP':'str', 'LATITUDE_LOCAL':'float', 
        'LONGITUDE_LOCAL':'float', 'NUM_SECAO':'str', 'SECAO_AGREGADORA':'str', 'SECAO_AGREGADA':'str',
        'id_unico':'str',
       }
    for file in files:
        df = pd.read_csv(file, dtype=dtype)
        dfs.append(df)
        
    df = pd.concat(dfs)
    return df

Agora vamos definir uma função que executa essas ações em ordem.

In [15]:
def geocode(cities, df, query_params, try_no, exception_list):
    geocode_each_city(cities, df, query_params, try_no, exception_list)
    df = concatenate_files(try_no).reset_index(drop=True) # reset_index para manter índices únicos após a concatenação
    return df

Enfim, rodamos a função, com uma sequência de combinações possíveis para as requisições.

In [16]:
all_queries = [
    
    ['LOCAL_VOTACAO', 'ENDERECO', 'LOCALIDADE_LOCAL_VOTACAO', 'SGL_UF', 'COUNTRY'],
    ['LOCAL_VOTACAO', 'CEP', 'LOCALIDADE_LOCAL_VOTACAO', 'SGL_UF', 'COUNTRY'],
    ['LOCAL_VOTACAO', 'LOCALIDADE_LOCAL_VOTACAO', 'SGL_UF', 'COUNTRY'],
    ['ENDERECO', 'BAIRRO_LOCAL_VOT', 'LOCALIDADE_LOCAL_VOTACAO', 'CEP', 'SGL_UF', 'COUNTRY'],
    ['ENDERECO', 'CEP', 'LOCALIDADE_LOCAL_VOTACAO', 'SGL_UF', 'COUNTRY'],
    ['ENDERECO', 'LOCALIDADE_LOCAL_VOTACAO', 'SGL_UF', 'COUNTRY'],
    
]

def run(all_queries, df, str_id):
    
    # Executa tentativas de geocodificação com
    # vários parâmetros possíveis.
    # str_id é um identificador do arquivo final
    
    cities = df.COD_LOCALIDADE_IBGE.unique()
    all_tries = range(1, len(all_queries) + 1)
    
    for query_params, try_no in zip(all_queries, all_tries):
        exception_list = []
        df = geocode(cities, df, query_params, try_no, exception_list)
        
    # A função salva apenas o último output, que é o estado dos dados após passar por todas as queries definidas
    df.to_csv("../data/geocode/geocode-" + str_id + '.csv', index=False)

In [ ]:
# Roda com um timestamp para identificar quando foi salvo
now = str(datetime.now())[:-7]
now = now.replace(' ','_')
now = now.replace(":","-")
run(all_queries, locais, now)

In [18]:
# Re-lê o arquivo para reiniciar trabalho
dtype = {
    'SGL_UF':'str', 'COD_LOCALIDADE_IBGE':'str', 'LOCALIDADE_LOCAL_VOTACAO':'str', 'ZONA':'str',
    'BAIRRO_ZONA_SEDE':'str', 'LATITUDE_ZONA':'float', 'LONGITUDE_ZONA':'float',
    'NUM_LOCAL':'str', 'SITUACAO_LOCAL':'str', 'TIPO_LOCAL':'str', 'LOCAL_VOTACAO':'str', 
    'ENDERECO':'str', 'BAIRRO_LOCAL_VOT':'str', 'CEP':'str', 'LATITUDE_LOCAL':'float', 
    'LONGITUDE_LOCAL':'float', 'NUM_SECAO':'str', 'SECAO_AGREGADORA':'str', 'SECAO_AGREGADA':'str',
    'id_unico':'str',
   }
locais = pd.read_csv("../data/geocode/geocode-" + now + "csv", dtype=dtype)

In [19]:
# O que temos, e em que nível de precisão?
locais.precision.value_counts(normalize=True)

ROOFTOP               0.418821
TSE                   0.236085
GEOMETRIC_CENTER      0.212389
APPROXIMATE           0.103929
RANGE_INTERPOLATED    0.028724
NO_VALUE              0.000053
Name: precision, dtype: float64

Para garantir um mínimo de precisão a partir de agora, vamos trabalhar apenas com dados classificados como `ROOFTOP` ou `TSE`. Um teste de amostra indicou que 94% das entradas `ROOFTOP` estão de fato corretas. As entradas `TSE` foram geolocalizadas manualmente pelo tribunal.

In [21]:
locais = locais[locais.precision.isin(['ROOFTOP', 'TSE'])]

É provável que algumas cidades pequenas fiquem sem nenhum local de votação localizado. Esses casos serão tratados posteriormente.

### 3. Verificar integridade dos dados

Agora, temos de fazer uma bateria de testes pare verificar se os dados estão corretos – se há entradas duplicadas e se os pontos estão no interior dos municípios onde deveriam estar, por exemplo.


De início, vamos verificar se há duplicatas. O Pandas consegue checar apenas strings. Assim, precisamos criar uma nova coluna, que combina a latitude e longitude para criar um elemento checável.

In [22]:
# Primeiro, aplicamos uma redução de precisão nos pontos encontrados. Queremos que eles sejam precisos até a quarta casa decimal.
# O Google, geralmente, retorna precisão até a sexta
locais["lat"] = locais.lat.round(4)
locais["lon"] = locais.lon.round(4)

In [23]:
def get_lat_lon_str(row):
    
    # Formata a latitude e longitude sem perder precisão na conversão
    
    lat = "{:.4f}".format(row.lat)
    lon = "{:.4f}".format(row.lon)
    
    lat_lon_str = lat + ', ' + lon
    
    return pd.Series({'lat_lon_str':lat_lon_str})

In [24]:
locais['lat_lon_str'] = locais.apply(get_lat_lon_str, axis=1)

In [25]:
# Conta as entradas duplicadas.
# Note o parêametro keep=False, que conta todos os itens e não apenas a primeira das repetições
locais.duplicated(subset='lat_lon_str', keep=False).value_counts()

False    52372
True      9622
dtype: int64

Há entradas com coordenadas geográficas duplicadas. Em alguns casos, isso pode ser correto: dois locais de votação que ocupam prédios diferentes de uma universidade, por exemplo. 

A notícia boa é que, como todas elas tem precisão `ROOFTOP` ou `TSE`, são **muito provalmente** duplicatas legítimas, possivelmente locais de votação categorizados de maneira distinta, mas que ocupam o mesmo prédio. Não temos condições de checar todas estas manualmente, então vamos assumir que estão corretas - os votos destas sessões serão agregados.

Outra checagem necessária é ver se todos os pontos que geolocalizamos se encontram nos limites de suas cidades. 

Primeiro, precisamos selecionar os mapas das cidades, que pegamos do IBGE.

In [26]:
city_maps = gpd.read_file("../data/geo/municipios-ibge/brasil-municipios/brasil-municipios.shp")
city_maps["NM_MUNICIP"] = city_maps.NM_MUNICIP.astype(str)
city_maps["CD_GEOCMU"] = city_maps.CD_GEOCMU.astype(str)

Para o restante do trabalho, precisamos ter certeza que tanto os mapas municipais quanto os pontos que geolocalizamos tem o mesmo CRS – ou seja, o mesmo sitema de coordenadas geográficas. Vamos em diante:

Agora, transformamos o objeto `locais` em um geodataframe, usando o CRS específico para o Google Maps.

In [27]:
geometry = [ Point( ( row.lon, row.lat ) ) for index, row in locais.iterrows() ]
locais = gpd.GeoDataFrame( locais, geometry = geometry )
locais.crs = {'init': 'EPSG:4326'}

Agora, transformamos os dados do IBGE para que fiquem nesse mesmo crs.

In [28]:
city_maps = city_maps.to_crs(locais.crs)

As próximas linhas de código determinam se os pontos são válidos, mantendo apenas os que estão dentro dos respectivos municípios. **Possivelmente, vamos perder alguns pontos.**


In [30]:
def validate_points(locais, city_maps):
    
# A função remove pontos que, por ventura, tenham sido localizados fora dos limites de suas respectivas cidades.

    to_concat = []
    count = 0
    
    for city_code in tqdm( city_maps.CD_GEOCMU.unique() ):
        flag = ''
        this_city = city_maps[city_maps.CD_GEOCMU == city_code].reset_index(drop=True)
        points = locais[locais.COD_LOCALIDADE_IBGE==city_code].reset_index(drop=True)

        city_name = this_city.loc[0, 'NM_MUNICIP']
        locais_validos = gpd.sjoin(points, this_city, how='inner', op='intersects')
        to_concat.append(locais_validos)
        
    locais_validos = pd.concat(to_concat).reset_index(drop=True)
    return locais_validos

Vamos derrubar todos os pontos que se encontram fora dos limites.

In [ ]:
locais = validate_points(locais, city_maps)

In [32]:
locais.to_csv("../data/locais/locais-validos.csv", index=False)

### 4. Ligar os votos ao local de votação

Vamos trabalhar com a variável `df` – ela é a lista **pura** dos locais de votação, sem os filtros aplicados previamente.

Há um problema: os códigos do TSE e do IBGE não são os mesmos, o que dificulta fazer o merge. Vamos usar um outro arquivo para fazer essa correspondência na tabela `locais`.

In [33]:
correspondencia = pd.read_csv("../data/votos/correspondencia-tse-ibge.csv", dtype={
    'GEOCOD_IBGE':'str',
    'COD_TSE':'str',
})

correspondencia["COD_TSE"] = correspondencia.COD_TSE.str.zfill(5)

Aqui, vamos criar um identificador único real para cada uma das seções. Isso é necessário porque o número identificador do TSE **não identifica a sessão de maneira única**.

In [34]:
base = base.merge(correspondencia[['GEOCOD_IBGE', 'COD_TSE']], 
                      left_on='COD_LOCALIDADE_IBGE', right_on='GEOCOD_IBGE')

Abaixo, lemos o arquivo com os boletins de urna do TSE.

In [35]:
dtype = {
    'DT_GERACAO':'str', 
    'HH_GERACAO':'str',
    'ANO_ELEICAO':'str', 
    'CD_PLEITO':'str', 
    'DT_PLEITO':'str', 
    'NUM_TURNO':'str', 
    'CD_ELEICAO':'str', 
    'DS_ELEICAO':'str', 
    'DT_ELEICAO':'str', 
    'SG_UF':'str', 
    'COD_TSE':'str', 
    'NM_MUNICIPIO':'str', 
    'ZONA':'str',
    'NUM_SECAO':'str', 
    'NR_LOCAL_VOTACAO':'str', 
    'CODIGO_CARGO':'str',
    'DS_CARGO_PERGUNTA':'str',
    'NR_PARTIDO':'str', 
    'SG_PARTIDO':'str', 
    'NM_PARTIDO':'str', 
    'QT_APTOS':'str',
    'QT_COMPARECIMENTO':'str',
    'QT_ABSTENCOES':'str',
    'CD_TIPO_URNA':'str', 
    'DS_TIPO_URNA':'str', 
    'CD_TIPO_VOTAVEL':'str', 
    'DS_TIPO_VOTAVEL':'str', 
    'NUM_VOTAVEL':'str',
    'NM_VOTAVEL':'str',
    'QTDE_VOTOS':'int',
    'NR_URNA_EFETIVADA':'str',
    'CD_CARGA_1_URNA_EFETIVADA':'str',
    'CD_CARGA_2_URNA_EFETIVADA':'str',
    'CD_FLASHCARD_URNA_EFETIVADA':'str',
    'DT_CARGA_URNA_EFETIVADA':'str',
    'DS_CARGO_PERGUNTA_SECAO':'str',
    'DS_AGREGADAS':'str',
    'DT_ABERTURA':'str',
    'DT_ENCERRAMENTO':'str',
    'QT_ELEITORES_BIOMETRIA_NH':'str',
    'NR_JUNTA_APURADORA':'str',
    'NR_TURMA_APURADORA':'str',
 
}

usecols = [
    'ANO_ELEICAO', 'CD_PLEITO', 'NUM_TURNO',
    'SG_UF', 'COD_TSE', 'NM_MUNICIPIO', 'ZONA', 'NUM_SECAO', 'NR_LOCAL_VOTACAO',
    'CODIGO_CARGO', 'DS_CARGO_PERGUNTA', 'NR_PARTIDO', 'SG_PARTIDO', 'QT_APTOS', 'QT_COMPARECIMENTO',
    'QT_ABSTENCOES', 'CD_TIPO_VOTAVEL', 'DS_TIPO_VOTAVEL', 'NUM_VOTAVEL', 'QTDE_VOTOS',
]

In [37]:
%%time

votos_por_secao = pd.read_csv("../data/votos/boletim-urna/boletins-concatenados-2-turno.csv",
                              encoding='Latin1', sep=',', header=0, names=usecols, 
                              dtype={col:'str' for col in usecols if 'QT' not in col}, 
                              usecols=usecols
                             )

CPU times: user 2min 15s, sys: 48.9 s, total: 3min 4s
Wall time: 4min 33s


In [39]:
def select_right_votes(df, cod_cargo, num_turno):
    
    # Seleciona os votos para a eleição - presidencial em primeiro turno, por exemplo
    
    votos_por_secao = df[(df.CODIGO_CARGO==str(cod_cargo)) & (df.NUM_TURNO==str(num_turno))]
    
    return votos_por_secao

In [40]:
def make_unique_id(df):
    
    # Cria um id único para cada sessão eleitoral
    
    df['ID_SECAO'] = df['COD_TSE'].astype(str) + '-' + df['ZONA'].astype(str) + '-' + df['NUM_SECAO'].astype(str)
    
    return df

In [41]:
def join_votes(df, votos_por_secao):
    
    # Reúne os votos da planilha de resultados com os locais de votação pré-catalogados
    
    votes = df.merge(votos_por_secao, on='ID_SECAO', how='inner', suffixes=["","__y"])
    votes = votes[[col for col in votes.columns if "__y" not in col]]
    
    return votes

In [42]:
def join_places(df, locais):
    
    # Reúne os locais de votação geolocalizados com os dados de votos.
    # Algumas seções devem se perder, já que nem todos os locais do país foram geolocalizados.
    
    votes = df.merge(locais, on='id_unico', how='inner', suffixes=["","__y"])
    votes = votes[[col for col in votes.columns if "__y" not in col]]
    
    return votes

In [43]:
def pivot_by_candidates(df, field):
    
    # Pega os dados que estão em formato long, com os dados de voto de cada candidato em uma linha
    # e transforma para o formato wide, com os votos de cada candidato em dado local como uma coluna do banco de dados
    
    temp_df = df.copy()
    
    columns = df.NUM_VOTAVEL.unique().tolist()
    votes = df.pivot_table(values='QTDE_VOTOS', index=field, columns='NUM_VOTAVEL', aggfunc='sum', fill_value=0)
    votes = votes.reset_index()
    # Duplicadas já contidas na pivot_table
    temp_df = temp_df.drop_duplicates(subset=field)
    
    votes = temp_df.merge(votes, on=field, how='inner')
    
    return votes

In [44]:
def groubpy_coords(df, cand_nos):
    
    # Agrupa os votos por coordenada geográfica, de modo a concentrar
    # locais de votação com ids distintos, mas que ocupam o mesmo espaço,
    # em um único ponto. É o caso de locais como "PUC - Bloco A" e "PUC - Bloco B"
    
    votes = df.copy()
    grouped_votes = df.groupby('lat_lon_str')[cand_nos].sum().reset_index()
    votes = votes.merge(grouped_votes, on='lat_lon_str', how='inner', suffixes=("__x","")) # Mantém os campos do groupby, que é o df da direita neste join, sem sufixo
    votes = votes.drop_duplicates(subset='lat_lon_str') # Remove duplicatas que se tornaram redundantes com o merge
    votes = votes[[col for col in votes.columns if '__x' not in col]]
    
    return votes

In [45]:
def run_votes(df, vote_data, locais, cod_cargo, num_turno):
    
    # Função que encapsula e roda as anteriores, com apenas um output.
    
    # Faz cópias para trabalhar no espectro local
    temp_df = df.copy()
    temp_vote_data = vote_data.copy()
    temp_locais = locais.copy()
    
    # Seleciona votos corretos
    votos_selecionados = select_right_votes(vote_data, cod_cargo, num_turno)
    
    # Cria códigos únicos para ligar os votos com suas respectivas sessões
    temp_df = make_unique_id(temp_df)
    votos_por_secao = make_unique_id(votos_selecionados)
    
    # Reune votes e seções
    votes = join_votes(temp_df, votos_selecionados)
    
    # Reúne as sessões e seus respectivos locais
    votes = join_places(votes, temp_locais)
    
    # Cria uma pivot table para colocar os votos na mesma linha
    votes = pivot_by_candidates(votes, "id_unico")
    
    # Agrupa por local
    votes = groubpy_coords(votes, votos_selecionados.NUM_VOTAVEL.unique())
    
    return votes

In [ ]:
votes = run_votes(base, votos_por_secao, locais, "1", "1")

### 5. Lidar com cidades que têm número insuficientes de locais
Para gerar os voronois, precisamos que cada cidade tenha ao menos quatro pontos válidos. Vamos remover os municípios que não atingem esse critério. Seus votos serão agrupados em um único local de votação, criado artificialmente no centróide da cidade.

In [47]:
def filter_incompletes(df):
    '''
    REMOVE TODOS OS LOCAIS DE CIDADES QUE NAO TENHAM AO MENOS QUATRO LOCAIS
    '''
    
    # Usa groupby filter para remover os dados das cidades que não desejamos
    votes = df.groupby('COD_TSE').filter(lambda x: x['GEOCOD_IBGE'].count() >= 4)
    return votes
    
    

In [48]:
def make_centroids(df, city_maps):
    '''
    ADICIONA OS CENTROIDES DE CADA UMA DAS CIDADES QUE FALTARAM
    '''
    new_df = pd.DataFrame()
    city_list = [city for city in city_maps.CDO_TSE if city not in df.COD_TSE.unique()]
    
    count = 0
    for city in city_list:
    # 1. Para cada uma das cidades ausentes no dataframe de votos, criar uma linha com o dados geográficos manualmente e dar append em um novo dataframe
    # Note que não vamos adicionar agora ao dataframe original, mas sim criar um novo que passará por um merge posteriormente.
        count += 1
        city_data = city_maps[city_maps.CDO_TSE==city].reset_index()
        city_data = city_data.loc[0]
        row = {
                 'ANO_ELEICAO': None,
                 'BAIRRO_LOCAL_VOT': None,
                 'BAIRRO_ZONA_SEDE': None,
                 'CDO_TSE': city_data.CDO_TSE,
                 'CD_GEOCMU': city_data.CD_GEOCMU,
                 'CEP': None,
                 'CODIGO_CARGO': None,
                 'COD_LOCALIDADE_IBGE': city_data.CD_GEOCMU,
                 'COD_TSE': city_data.CDO_TSE,
                 'COUNTRY': "Brasil",
                 'DATA_GERACAO': None,
                 'DESCRICAO_CARGO': None,
                 'DESCRICAO_ELEICAO': None,
                 'ENDERECO': "Centróide de " + city_data.NM_MUNICIP,
                 'GEOCOD_IBGE': city_data.CD_GEOCMU,
                 'HORA_GERACAO': None,
                 'ID_SECAO': 'FAKE' + str(count).zfill(4),
                 'LATITUDE_LOCAL': None,
                 'LATITUDE_ZONA': None,
                 'LOCALIDADE_LOCAL_VOTACAO': city_data.NM_MUNICIP,
                 'LOCAL_VOTACAO': "Centróide de " + city_data.NM_MUNICIP,
                 'LONGITUDE_LOCAL': None,
                 'LONGITUDE_ZONA': None,
                 'NM_MUNICIP': city_data.NM_MUNICIP,
                 'NOME_MUNICIPIO': city_data.NM_MUNICIP,
                 'NUM_LOCAL': None,
                 'NUM_SECAO': None,
                 'NUM_TURNO': None,
                 'NUM_VOTAVEL': None,
                 'QTDE_VOTOS': None,
                 'SECAO_AGREGADA': None,
                 'SECAO_AGREGADORA': None,
                 'SGL_UF': city_data.UF,
                 'SIGLA_UE': city_data.UF,
                 'SIGLA_UF': city_data.UF,
                 'SITUACAO_LOCAL': None,
                 'TIPO_LOCAL': None,
                 'UF': city_data.UF,
                 'ZONA': 'FAKE' + str(count).zfill(4),
                 'fetched_address': "Centróide de " + city_data.NM_MUNICIP,
                 'geometry': city_data.geometry.centroid,
                 'id_unico': 'FAKE' + str(count).zfill(4),
                 'index_right': None,
                 'lat': round(city_data.geometry.centroid.coords[0][1], 4),
                 'lat_lon_str': None,
                 'local_unico': None,
                 'lon': round(city_data.geometry.centroid.coords[0][0], 4),
                 'precision': "CITY_CENTROID"
        }        
        new_df = new_df.append(row, ignore_index=True)
    
    new_df['lat_lon_str'] = new_df.apply(get_lat_lon_str, axis=1)
    return new_df

In [49]:
def get_centroid_votes(df, vote_data, cod_cargo, num_turno, city_maps):
    '''
    PEGA OS VOTOS DA CIDADE TODA, QUANDO ELA NÃO ESTÁ CONTEMPLADA NOS LOCAIS DE VOTAÇÃO
    '''
    centroids = df.copy()
    vote_data_temp = vote_data.copy()
    city_list = [city for city in city_maps.CDO_TSE if city in df.COD_TSE.unique()]
    
    # Seleciona votos corretos
    vote_data_temp = select_right_votes(vote_data_temp, cod_cargo, num_turno)
    vote_data_temp = vote_data_temp[vote_data_temp.COD_TSE.isin(city_list)]
    
    # Agrupa dados dos votos de forma correta
    vote_data_temp = pivot_by_candidates(vote_data_temp, "COD_TSE")
    
    # Faz o merge
    centroids = centroids.merge(vote_data_temp, on='COD_TSE', how='inner', suffixes=("","__y")) # Note que aqui, quando não estamos trabalhando com o país todo, cidades sem votos registrados podem desaparecer.
    centroids = centroids[[col for col in centroids.columns if "__y" not in col]]
    
    # Derruba duplicatas
    centroids = centroids.drop_duplicates(subset='COD_TSE')
    
    return centroids

In [50]:
def concatenate(df, new_df):
    
    # Encapsula função built-in apenas por conveniência
    
    return pd.concat([df, new_df])

In [51]:
def clean_cols(df):
    
    # Derruba colunas desnecessárioas
    
    to_drop = [
        'ANO_ELEICAO', 'BAIRRO_LOCAL_VOT', 'BAIRRO_ZONA_SEDE', 'CDO_TSE',
       'CD_GEOCMU', 'CEP', 'CODIGO_CARGO', 'COD_LOCALIDADE_IBGE',
       'COUNTRY', 'DATA_GERACAO', 'DESCRICAO_CARGO', 'DESCRICAO_ELEICAO',
       'ENDERECO', 'HORA_GERACAO', 'ID_SECAO', 'LATITUDE_LOCAL',
       'LATITUDE_ZONA', 'LOCALIDADE_LOCAL_VOTACAO', 'LOCAL_VOTACAO',
       'LONGITUDE_LOCAL', 'LONGITUDE_ZONA', 'NOME_MUNICIPIO',
       'NUM_LOCAL', 'NUM_SECAO', 'NUM_TURNO', 'NUM_VOTAVEL', 'QTDE_VOTOS',
       'SECAO_AGREGADA', 'SECAO_AGREGADORA', 'SGL_UF', 'SIGLA_UE', 'SIGLA_UF',
       'SITUACAO_LOCAL', 'TIPO_LOCAL', 'index_right', 'precision',
    ]
    
    votes = df.drop(to_drop, axis=1)
    return votes

In [52]:
def run_centroids_op(df, vote_data, cod_cargo, num_turno, city_maps):
    
    # Encapsula as funções definidas anteriormente
    
    votes = filter_incompletes(df)
    centroids = make_centroids(votes, city_maps)
    centroids = get_centroid_votes(centroids, vote_data, cod_cargo, num_turno, city_maps)
    votes = concatenate(votes, centroids)
    votes = gpd.GeoDataFrame(votes, geometry=votes.geometry)
    votes = clean_cols(votes)
    
    return votes

In [ ]:
votes = run_centroids_op(votes, votos_por_secao, "1", "1", city_maps)

Agora, de posse dos votos, podemos fazer calculos do percentual de votos de cada candidato em cada voronoi.

In [54]:
def get_percentage_votes(row):
    
    # Calcula o percentual dos votos de cada candidato
        
    all_nums = [col for col in list(row.keys()) if col.isnumeric()] # Votos válidos
    party_nums = [col for col in list(row.keys()) if col.isnumeric() and col not in ['95', '96']] # Brancos e nulos
    
    total_votes = row[party_nums].sum() # Total de votos válidos
    total_votes_with_nulls = row[all_nums].sum() # Total de votos incluindo brancos e nulos
    
    # Dicionário com valores para serem preenchidos
    series = {col + "_PP":None for col in all_nums}
    
    for num in all_nums:
        
        # Calcula percentual dos votos válidos
        if num not in ['95', '96']:
            try:
                percentage = round(row[num] / total_votes, 3)
                series[num + "_PP"] = percentage
            # Se determinado candidato não tiver voto algum neste local, capturar a exceção e preencher com nan
            except ZeroDivisionError as e:
                series[num + "_PP"] = np.nan
        
        # Aqui, crio variáveis com o percentual de brancos e nulos
        # O else é necessário para fazer o cálculo com uma variável diferente
        else:
            try:
                percentage = round(row[num] / total_votes_with_nulls, 3)
                series[num + "_PP"] = percentage
                
            # Do mesmo modo que acima, captura exceção caso não haja brancos ou nulos no local
            except ZeroDivisionError as e:
                series[num + "_PP"] = np.nan

    return pd.Series(series)

In [55]:
votes = pd.concat([votes, votes.apply(get_percentage_votes, axis=1)], axis=1)

### 6. Desenhar diagrama de Voronoi

Com os locais listados, resta desenhar os diagramas de Voronoi. Vamos usar um método baseado [neste](https://github.com/ipython-books/cookbook-2nd-code/blob/master/chapter14_graphgeo/05_voronoi.ipynb) material. O output da função são diversos arquivos .shp, um para cada cidade.

In [ ]:
def voronoi_finite_polygons_2d(vor, radius=None):
    """Reconstruct infinite Voronoi regions in a
    2D diagram to finite regions.
    Source:
    https://stackoverflow.com/a/20678647/1595060
    """
    if vor.points.shape[1] != 2:
        raise ValueError("Requires 2D input")
    new_regions = []
    new_vertices = vor.vertices.tolist()
    center = vor.points.mean(axis=0)
    if radius is None:
        radius = vor.points.ptp().max()
    # Construct a map containing all ridges for a
    # given point
    all_ridges = {}
    for (p1, p2), (v1, v2) in zip(vor.ridge_points,
                                  vor.ridge_vertices):
        #print(p1, p2)
        all_ridges.setdefault(
            p1, []).append((p2, v1, v2))
        all_ridges.setdefault(
            p2, []).append((p1, v1, v2))
    # Reconstruct infinite regions
    for p1, region in enumerate(vor.point_region):
        vertices = vor.regions[region]
        if all(v >= 0 for v in vertices):
            # finite region
            new_regions.append(vertices)
            continue
        # reconstruct a non-finite region
        ridges = all_ridges[p1]
        new_region = [v for v in vertices if v >= 0]
        for p2, v1, v2 in ridges:
            if v2 < 0:
                v1, v2 = v2, v1
            if v1 >= 0:
                # finite ridge: already in the region
                continue
            # Compute the missing endpoint of an
            # infinite ridge
            t = vor.points[p2] - \
                vor.points[p1]  # tangent
            t /= np.linalg.norm(t)
            n = np.array([-t[1], t[0]])  # normal
            midpoint = vor.points[[p1, p2]]. \
                mean(axis=0)
            direction = np.sign(
                np.dot(midpoint - center, n)) * n
            far_point = vor.vertices[v2] + \
                direction * radius
            new_region.append(len(new_vertices))
            new_vertices.append(far_point.tolist())
        # Sort region counterclockwise.
        vs = np.asarray([new_vertices[v]
                         for v in new_region])
        c = vs.mean(axis=0)
        angles = np.arctan2(
            vs[:, 1] - c[1], vs[:, 0] - c[0])
        new_region = np.array(new_region)[
            np.argsort(angles)]
        new_regions.append(new_region.tolist())
    return new_regions, np.asarray(new_vertices)

In [ ]:
def reverse_coords(polygons):
    
    # Reverte as coordenadas que ficaram invertidas após a execução anterior
    
    temp = []
    for polygon in polygons:
        temp_polygon = []
        for vertice in polygon:
            vertice = [vertice[1], vertice[0]]
            temp_polygon.append(vertice)
        temp_polygon = np.array(temp_polygon)
        temp.append(temp_polygon)
    
    return temp

In [ ]:
def merge_polygons(df):
    '''
    Função que leva um geo_df como parâmetro e faz merge em todos os polígonos
     que compartilham um mesmo id. Isso é necessário porque, depois de recortar um polígono voronoi
     contra um contorno de cidade convexo, às vezes acabamos com dois ou mais polígonos para
     um único local.
    
     INPUT: geo_df após calcular a interseção
     SAÍDA: geo_df com polígonos mesclados
    '''
    # Para cada item...
    for index, row in df.iterrows():
        # Seleciona todos os polígonos com o mesmo id
        temp = df[df.id_unico == row.id_unico]
        # Se houver mais de um polígono com o mesmo id
        if temp.shape[0] > 1:
            # Junte todos em apenas um
            new_polygon = temp.unary_union
            # E subistitua no df original
            df.at[index, 'geometry'] = new_polygon
    
    # Depois, derrube as duplicatas desnecessárias
    new_df = df.drop_duplicates(subset='id_unico')
    return new_df

In [ ]:
def make_voronoi_maps(cities, locais, city_maps, exceptions):
    '''
    INPUT:
    cities = array com códigos IBGE
    locais = um geo_df com os pontos de cada local de votação
    city_maps = geo_df com limites de cidades do IBGE
    exeptions = lista vazia que vai ser populada com possíveis erros
    '''
    count = 0
    for index, city in enumerate(tqdm(cities)):
        #print('Making city', city, '-', len(cities) - index, 'cities to go')
        try:
            # Pega o mapa a partir dos arquivos do IBGE
            outline = city_maps[city_maps.CD_GEOCMU==city].reset_index()

            # Seleciona e salva apenas os pontos correspondentes 
            points = locais[locais.GEOCOD_IBGE==city].reset_index()
            #print("Points selected and saved")

            # Se não houver ao menos três pontos, é impossível computar um voronoi.
            # Nesse caso, em vez de um voronoi, salvamos o próprio outline da cidade
            if points.shape[0] < 4 and points.shape[0] != 1:
                print("Isso não devia acontecer. Caos!")
                print(points.shape[0], 'pontos detectados')
                print(index, city)
                return

            if points.shape[0] == 1:
                count += 1
                polygons_df = points.copy()
                polygons_df.geometry = outline.geometry
                polygons_df.to_file('../data/geo/voronois-merge/' + city + '.shp')

            else:
                # Salva os pontos encontrados
                points.to_file('../data/geo/pontos-cidade/' + city + '.shp')

                # Isola as latitudes e longitudes em arrays específicos
                lat = points.lat
                lon = points.lon

                # Usa o pacote voronoi do scipy.spatial
                vor = spatial.Voronoi(np.c_[lat, lon])
                # Calcula limites finitos para os voronois retornados pelo scipy
                regions, vertices = voronoi_finite_polygons_2d(vor, 10000)
                # Cria um array com as coordenadas
                polygons = [vertices[region] for region in regions]
                # Reverte para o formato lon/lat
                #print("Reversing coordinates")
                polygons = reverse_coords(polygons)
                # Transforma o np.array em um polígono do shapely
                polygons = [Polygon(polygon) for polygon in polygons]

                # Cria um df com o polígono de cada pontos
                polygons_df = points.copy()
                polygons_df.geometry = polygons

                # Salva uma versão pré-clipar
                polygons_df.crs = {'init': 'EPSG:4326'}
                polygons_df.to_file('../data/geo/voronois-pre-clip/' + city + '.shp')

                # Corta os polígonos de voronoi no outline da cidade
                # O overlay padrão do geopandas não parece funcionar, gerando alguns bugs inesperados com float precision.
                # Vamos ter que fazer dentro de um loop usando a função base do shapely, que opera em apenas um polígono de cada vez.
                new_geoseries = []
                poly_list = polygons_df.geometry.tolist()
                outline_geom = outline.loc[0, 'geometry'].buffer(0) # O buffer(0) corrige self-intersections em quatro municípios
                for index, poly in enumerate(poly_list):
                    if not (poly.is_valid and outline_geom.is_valid):
                        print("Problem with geometry validity.")
                        print("poly.is_valid:", poly.is_valid, "outline_geom.is_valid:", outline_geom.is_valid)
                    intersection = poly.intersection(outline_geom)
                    new_geoseries.append(intersection)

                # Salva uma versão clipada
                polygons_df.geometry = new_geoseries
                polygons_df.crs = {'init': 'EPSG:4326'}
                polygons_df.to_file('../data/geo/voronois-clipados/' + city + '.shp')


                # Após fazer a clipagem, criam-se polígonos isolados, que pertenciam a um voronoi mas acabaram cortados pelo relevo da cidade.
                # Eles mantém os campos id_unico e lat_lon_str do voronoi, porém. Podemos reagrupá-los a partir daí.
                polygons_df = merge_polygons(polygons_df)
                polygons_df.crs = {'init': 'EPSG:4326'}
                polygons_df.to_file('../data/geo/voronois-merge/' + city + '.shp')
                          
        # Captura exceção, printa no console e salva o código da cidade em lista
        except Exception as e:
            exc_type, exc_obj, tb = sys.exc_info()
            f = tb.tb_frame
            lineno = tb.tb_lineno
            filename = f.f_code.co_filename
            linecache.checkcache(filename)
            line = linecache.getline(filename, lineno, f.f_globals)
            exceptions.append(city)
            print("Error", e, "on city code", city)
            print('EXCEPTION IN', lineno, line)
            print()

In [ ]:
# Roda a função com todos os códigos de cidade.
# Note que as duplicatas não são derrubadas in place porque elas são necessárias para reunir votos e voronois posteriormente.
cities = votes.GEOCOD_IBGE.unique()
exceptions = []
make_voronoi_maps( cities, votes, city_maps, exceptions)

Com os arquivos já salvos, precisamos ler e concatenar todos.

In [ ]:
files = glob.glob("../data/geo/voronois-merge/*.shp")

In [ ]:
voronois = pd.concat([gpd.read_file(file) for file in files])

Aqui, criamos um dicionário de dados com os campos do shapefile.

In [ ]:
columns = {
 'GEOCOD_IBG': 'COD_IBGE',
 'lat_lon_st': 'geom_str',
 'local_unic': 'loc_unico',
}

In [ ]:
voronois = voronois.rename(columns=columns)

### 6. Ligar limites dos setores censitários aos dados
Agora vamos ligar os polígonos dos setores censitários aos seus respectivos dados.

Primeiro, precisamos ler as shapefiles do Censo 2010.

In [ ]:
def get_dirs(pattern):
    '''
    Essa função usa glob e um loop para criar uma lista
    dos arquivos que precisamos. Eles estão salvos em uma
    estrutura de árvore no diretório, logo um glob simples
    não bastaria. De posse da lista, podemos ler os arquivos
    todos ao mesmo tempo, simplesmente usando a função built-in
    pd.concat([gpd.read(file) for file in files])
    '''
    directories = glob.glob(pattern)
    files = []
    for directory in directories:
        file = glob.glob(directory + '/*.shp')
        files.append(file[0])
        
    return files

Feito isso, precisamos colocar os dados do censo no geodataframe – por enquanto, os shapefiles dos setores têm apenas as inforamções geográficas.

Os arquivos de dados do censo são dividios em vários formatos, cada um com estatísticas específicas. Vamos ler e padronizar estes grupos de arquivos dentro de um loop.

**1. Basico_UF:** informações geográficas e quantidade de residentes em domicílios particulares permanentes.

**2. Domicilio01_UF:** Informações sobre o responsável do domicílio. Vamos usar para extrair dados de mulheres chefes de família.

**3. Domicilio02_UF:** Informações sobre o status do domicílio: é próprio, alugado, cedido?

**4. Pessoa01_UF:** Informações sobre a alfabetização dos habitantes.

**5. Pessoa03_UF:** Informações sobre a raça dos habitantes.

**6. Pessoa13_UF:** Informações sobre a idade dos habitantes.


In [1]:
def read_data(pattern, correspondence, df_list):
    '''
    Essa função abre cada um dos arquivos da lista retornada por get_dirs(pattern)
    e trata elas de acordo com os parâmetros patterns e correspondence, que serão definidos
    antes da chamada da função. Eles devem contar o dicionário de dados do Censo 2010, já
    que o cabeçalho dos arquivos contém apenas valores numéricos.
    '''
    files = glob.glob(pattern)
    dtype = {
            'Cod_setor':'str',                       
            'Cod_Grandes Regiões':'str',                            
            'Cod_UF':'str',                                        
            'Cod_meso':'str',                                       
            'Cod_micro':'str',                                      
            'Cod_RM':'str',                                         
            'Cod_municipio':'str',                                  
            'Cod_distrito':'str',                                   
            'Cod_subdistrito':'str',                                
            'Cod_bairro':'str',                                     
            'Situacao_setor':'str',
            'Tipo_setor':'str',                                  
            }
    fields = list(dtype.keys())
    fields.extend(list(correspondence.keys()))
    df = pd.concat([pd.read_excel(file, dtype=dtype, usecols_excel=fields) for file in files])
    df = df.rename(columns=correspondence)
    df_list.append(df)

Os arquivos do censo tem campos diferentes e precisam passar por um merge/join para conversar um com os outros. Vamos fazer isso usando a função `reduce`. Uma vez que os dados estejam unidos, é preciso fazer alguns filtros para evitar campos pesados e desnecessários. A função `merge_censo(df_list)` faz tudo isso de uma vez só.

In [ ]:
def merge_censo(df_list):
    dados_censo = reduce(lambda  left,right: pd.merge(left,right,on='Cod_setor', how='inner', suffixes=('','__y')), df_list)
    
    # Tira colunas duplicadas pelo merge
    to_keep = [column for column in dados_censo.columns if '__y' not in column]
    dados_censo = dados_censo[to_keep]
    
    # Usa um regex com lookahead negativo para manter apenas as colunas que não começam com V\d{3}
    # dados_censo = dados_censo.filter(regex='^(?!V\d{3}).+', axis=1)
    
    return dados_censo

Há um problema que precisa ser contornado: para determinadas variáveis, em certos setores censitários, o IBGE decidiu anonimizar os dados: em vez de números, os campo têm 'X'. São setores muito pequenos e revelar os dados poderia expor populações sensíveis. Vamos tratá-los como **ZERO**. Como havia um 'X' nesses campos, o Pandas pensa que está lidando com strings em todas as colunas. Não é o caso. Precisamos converter.

In [ ]:
def handle_anom_data(df, correspondences):
    # Remove X e substitui por zero
    dados_censo = df.replace('X', 0)
    
    # Pega os campos que deveriam ser numéricos e transforma numa lista de headers.
    num_fields = []
    for item in correspondences:
        num_fields.extend(item.values())
        
    # Trasnforma estes campos em dtypes numéricos
    dados_censo[num_fields] = dados_censo[num_fields].apply(pd.to_numeric, axis=0)
    
    return dados_censo

A função abaixo encapsula todo esse processo e salva o output para um arquivo `.csv`.

In [ ]:
def run_censo_concat():
    
    # 1. Lê arquivos dos setores censitários
    files = get_dirs("../data/geo/setores-censitarios-ibge/*")

    # 2. Concatena todos os arquivos de setores em apenas um df
    setores = pd.concat([gpd.read_file(file) for file in files])
  
    # 3. Define parâmetros estáticos para as chamadas de função posteriores
    patterns = [
            "../data/censo/files/Basico_*", 
           '../data/censo/files/Domicilio01_*',
           '../data/censo/files/Domicilio02_*',
           '../data/censo/files/Pessoa01_*',
           '../data/censo/files/Pessoa03_*',
           '../data/censo/files/Pessoa13_*',
           '../data/censo/files/Pessoa11_*',
           '../data/censo/files/Pessoa12_*',
            ]
   
    correspondences = [
    
        # ARQUIVO_BÁSICO_UF.XLS
        {
            'V001':'dom_part_permanentes',
            'V002':'moradores_dom_part_permanentes',
            'V003':'media_num_moradores_dom_part_permanentes',
            'V004':'Var_num_moradores_dom_part_permanentes',
            'V005':'media_renda_nom_mensal_responsaVeis_dom_part_permanentes',
            'V006':'Var_renda_nom_mensal_responsaVeis_dom_part_permanentes',
            'V007':'media_renda_nom_mensal_responsaVeis_dom_part_permanentes_com_rendimento',
            'V008':'Var_media_renda_nom_mensal_responsaVeis_dom_part_permanentes_com_rendimento',
            'V009':'media_renda_nom_mensal_pessoas_com_10_anos_ou_mais_com_ou_sem_rendimento',
            'V010':'Var_renda_nom_mensal_pessoas_com_10_anos_ou_mais_com_ou_sem_rendimento',
            'V011':'media_renda_nom_mensal_pessoas_com_10_anos_ou_mais_com_rendimento',
            'V012':'Var_renda_nom_mensal_pessoas_com_10_anos_ou_mais_com_rendimento',
        },
        # ARQUIVO_DOMICÍLIO01_UF.XLS
        {
            'V081':'domicílios_particulares_permanentes_com_mulher_responsável_e_mais_1_morador',
            'V082':'domicílios_particulares_permanentes_com_mulher_responsável_e_mais_2_moradores',
            'V083':'domicílios_particulares_permanentes_com_mulher_responsável_e_mais_3_moradores',
            'V084':'domicílios_particulares_permanentes_com_mulher_responsável_e_mais_4_moradores',
            'V085':'domicílios_particulares_permanentes_com_mulher_responsável_e_mais_5_moradores',
            'V086':'domicílios_particulares_permanentes_com_mulher_responsável_e_mais_6_ou_mais_moradores',
            'V087':'domicílios_particulares_permanentes_com_mulher_responsáVel_e_sem_outro_morador',
        },
        # ARQUIVO_DOMICÍLIO02_UF.XLS
        {
            'V001':'moradores_em_domicílios_particulares_e_domicílios_coletivos',
            'V002':'moradores_em_domicílios_particulares_permanentes',
            'V003':'moradores_em_domicílios_particulares_permanentes_do_tipo_casa',
            'V004':'moradores_em_domicílios_particulares_permanentes_do_tipo_casa_de_vila_ou_em_condomínio',
            'V005':'moradores_em_domicílios_particulares_permanentes_do_tipo_apartamento',
            'V006':'moradores_em_domicílios_particulares_permanentes_próprios_e_quitados',
            'V007':'moradores_em_domicílios_particulares_permanentes_próprios_e_em_aquisição',
            'V008':'moradores_em_domicílios_particulares_permanentes_alugados',
            'V009':'moradores_em_domicílios_particulares_permanentes_cedidos_por_empregador',
            'V010':'moradores_em_domicílios_particulares_permanentes_cedidos_de_outra_forma',
            'V011':'moradores_em_domicílios_particulares_permanentes_com_outra_condição_de_ocupação',
        },
        # ARQUIVO_PESSOA01_UF
        {
            'V001':'pessoas_alfabetizadas_com_5_ou_mais_anos_de_idade',
            'V013':'pessoas_alfabetizadas_com_16_anos_de_idade',
            'V014':'pessoas_alfabetizadas_com_17_anos_de_idade',
            'V015':'pessoas_alfabetizadas_com_18_anos_de_idade',
            'V016':'pessoas_alfabetizadas_com_19_anos_de_idade',
            'V017':'pessoas_alfabetizadas_com_20_anos_de_idade',
            'V018':'pessoas_alfabetizadas_com_21_anos_de_idade',
            'V019':'pessoas_alfabetizadas_com_22_anos_de_idade',
            'V020':'pessoas_alfabetizadas_com_23_anos_de_idade',
            'V021':'pessoas_alfabetizadas_com_24_anos_de_idade',
            'V022':'pessoas_alfabetizadas_com_25_anos_de_idade',
            'V023':'pessoas_alfabetizadas_com_26_anos_de_idade',
            'V024':'pessoas_alfabetizadas_com_27_anos_de_idade',
            'V025':'pessoas_alfabetizadas_com_28_anos_de_idade',
            'V026':'pessoas_alfabetizadas_com_29_anos_de_idade',
            'V027':'pessoas_alfabetizadas_com_30_anos_de_idade',
            'V028':'pessoas_alfabetizadas_com_31_anos_de_idade',
            'V029':'pessoas_alfabetizadas_com_32_anos_de_idade',
            'V030':'pessoas_alfabetizadas_com_33_anos_de_idade',
            'V031':'pessoas_alfabetizadas_com_34_anos_de_idade',
            'V032':'pessoas_alfabetizadas_com_35_anos_de_idade',
            'V033':'pessoas_alfabetizadas_com_36_anos_de_idade',
            'V034':'pessoas_alfabetizadas_com_37_anos_de_idade',
            'V035':'pessoas_alfabetizadas_com_38_anos_de_idade',
            'V036':'pessoas_alfabetizadas_com_39_anos_de_idade',
            'V037':'pessoas_alfabetizadas_com_40_anos_de_idade',
            'V038':'pessoas_alfabetizadas_com_41_anos_de_idade',
            'V039':'pessoas_alfabetizadas_com_42_anos_de_idade',
            'V040':'pessoas_alfabetizadas_com_43_anos_de_idade',
            'V041':'pessoas_alfabetizadas_com_44_anos_de_idade',
            'V042':'pessoas_alfabetizadas_com_45_anos_de_idade',
            'V043':'pessoas_alfabetizadas_com_46_anos_de_idade',
            'V044':'pessoas_alfabetizadas_com_47_anos_de_idade',
            'V045':'pessoas_alfabetizadas_com_48_anos_de_idade',
            'V046':'pessoas_alfabetizadas_com_49_anos_de_idade',
            'V047':'pessoas_alfabetizadas_com_50_anos_de_idade',
            'V048':'pessoas_alfabetizadas_com_51_anos_de_idade',
            'V049':'pessoas_alfabetizadas_com_52_anos_de_idade',
            'V050':'pessoas_alfabetizadas_com_53_anos_de_idade',
            'V051':'pessoas_alfabetizadas_com_54_anos_de_idade',
            'V052':'pessoas_alfabetizadas_com_55_anos_de_idade',
            'V053':'pessoas_alfabetizadas_com_56_anos_de_idade',
            'V054':'pessoas_alfabetizadas_com_57_anos_de_idade',
            'V055':'pessoas_alfabetizadas_com_58_anos_de_idade',
            'V056':'pessoas_alfabetizadas_com_59_anos_de_idade',
            'V057':'pessoas_alfabetizadas_com_60_anos_de_idade',
            'V058':'pessoas_alfabetizadas_com_61_anos_de_idade',
            'V059':'pessoas_alfabetizadas_com_62_anos_de_idade',
            'V060':'pessoas_alfabetizadas_com_63_anos_de_idade',
            'V061':'pessoas_alfabetizadas_com_64_anos_de_idade',
            'V062':'pessoas_alfabetizadas_com_65_anos_de_idade',
            'V063':'pessoas_alfabetizadas_com_66_anos_de_idade',
            'V064':'pessoas_alfabetizadas_com_67_anos_de_idade',
            'V065':'pessoas_alfabetizadas_com_68_anos_de_idade',
            'V066':'pessoas_alfabetizadas_com_69_anos_de_idade',
            'V067':'pessoas_alfabetizadas_com_70_anos_de_idade',
            'V068':'pessoas_alfabetizadas_com_71_anos_de_idade',
            'V069':'pessoas_alfabetizadas_com_72_anos_de_idade',
            'V070':'pessoas_alfabetizadas_com_73_anos_de_idade',
            'V071':'pessoas_alfabetizadas_com_74_anos_de_idade',
            'V072':'pessoas_alfabetizadas_com_75_anos_de_idade',
            'V073':'pessoas_alfabetizadas_com_76_anos_de_idade',
            'V074':'pessoas_alfabetizadas_com_77_anos_de_idade',
            'V075':'pessoas_alfabetizadas_com_78_anos_de_idade',
            'V076':'pessoas_alfabetizadas_com_79_anos_de_idade',
            'V077':'pessoas_alfabetizadas_com_80_anos_ou_mais_de_idade',
        },
        # ARQUIVO_PESSOA03_UF.XLS
        {
            'V001':'pessoas_residentes',
            'V002':'pessoas_residentes_e_cor_ou_raça_branca',
            'V003':'pessoas_residentes_e_cor_ou_raça_preta',
            'V004':'pessoas_residentes_e_cor_ou_raça_amarela',
            'V005':'pessoas_residentes_e_cor_ou_raça_parda',
            'V006':'pessoas_residentes_e_cor_ou_raça_indígenas',
        },
        # ARQUIVO_PESSOA13_UF.XLS
        {
            'V022':'pessoas_com_menos_de_1_ano_de_idade',
            'V035':'pessoas_de_1_ano_de_idade',
            'V036':'pessoas_com_2_anos_de_idade',
            'V037':'pessoas_com_3_anos_de_idade',
            'V038':'pessoas_com_4_anos_de_idade',
            'V039':'pessoas_com_5_anos_de_idade',
            'V040':'pessoas_com_6_anos_de_idade',
            'V041':'pessoas_com_7_anos_de_idade',
            'V042':'pessoas_com_8_anos_de_idade',
            'V043':'pessoas_com_9_anos_de_idade',
            'V044':'pessoas_com_10_anos_de_idade',
            'V045':'pessoas_com_11_anos_de_idade',
            'V046':'pessoas_com_12_anos_de_idade',
            'V047':'pessoas_com_13_anos_de_idade',
            'V048':'pessoas_com_14_anos_de_idade',
            'V049':'pessoas_com_15_anos_de_idade',
            'V050':'pessoas_com_16_anos_de_idade',
            'V051':'pessoas_com_17_anos_de_idade',
            'V052':'pessoas_com_18_anos_de_idade',
            'V053':'pessoas_com_19_anos_de_idade',
            'V054':'pessoas_com_20_anos_de_idade',
            'V055':'pessoas_com_21_anos_de_idade',
            'V056':'pessoas_com_22_anos_de_idade',
            'V057':'pessoas_com_23_anos_de_idade',
            'V058':'pessoas_com_24_anos_de_idade',
            'V059':'pessoas_com_25_anos_de_idade',
            'V060':'pessoas_com_26_anos_de_idade',
            'V061':'pessoas_com_27_anos_de_idade',
            'V062':'pessoas_com_28_anos_de_idade',
            'V063':'pessoas_com_29_anos_de_idade',
            'V064':'pessoas_com_30_anos_de_idade',
            'V065':'pessoas_com_31_anos_de_idade',
            'V066':'pessoas_com_32_anos_de_idade',
            'V067':'pessoas_com_33_anos_de_idade',
            'V068':'pessoas_com_34_anos_de_idade',
            'V069':'pessoas_com_35_anos_de_idade',
            'V070':'pessoas_com_36_anos_de_idade',
            'V071':'pessoas_com_37_anos_de_idade',
            'V072':'pessoas_com_38_anos_de_idade',
            'V073':'pessoas_com_39_anos_de_idade',
            'V074':'pessoas_com_40_anos_de_idade',
            'V075':'pessoas_com_41_anos_de_idade',
            'V076':'pessoas_com_42_anos_de_idade',
            'V077':'pessoas_com_43_anos_de_idade',
            'V078':'pessoas_com_44_anos_de_idade',
            'V079':'pessoas_com_45_anos_de_idade',
            'V080':'pessoas_com_46_anos_de_idade',
            'V081':'pessoas_com_47_anos_de_idade',
            'V082':'pessoas_com_48_anos_de_idade',
            'V083':'pessoas_com_49_anos_de_idade',
            'V084':'pessoas_com_50_anos_de_idade',
            'V085':'pessoas_com_51_anos_de_idade',
            'V086':'pessoas_com_52_anos_de_idade',
            'V087':'pessoas_com_53_anos_de_idade',
            'V088':'pessoas_com_54_anos_de_idade',
            'V089':'pessoas_com_55_anos_de_idade',
            'V090':'pessoas_com_56_anos_de_idade',
            'V091':'pessoas_com_57_anos_de_idade',
            'V092':'pessoas_com_58_anos_de_idade',
            'V093':'pessoas_com_59_anos_de_idade',
            'V094':'pessoas_com_60_anos_de_idade',
            'V095':'pessoas_com_61_anos_de_idade',
            'V096':'pessoas_com_62_anos_de_idade',
            'V097':'pessoas_com_63_anos_de_idade',
            'V098':'pessoas_com_64_anos_de_idade',
            'V099':'pessoas_com_65_anos_de_idade',
            'V100':'pessoas_com_66_anos_de_idade',
            'V101':'pessoas_com_67_anos_de_idade',
            'V102':'pessoas_com_68_anos_de_idade',
            'V103':'pessoas_com_69_anos_de_idade',
            'V104':'pessoas_com_70_anos_de_idade',
            'V105':'pessoas_com_71_anos_de_idade',
            'V106':'pessoas_com_72_anos_de_idade',
            'V107':'pessoas_com_73_anos_de_idade',
            'V108':'pessoas_com_74_anos_de_idade',
            'V109':'pessoas_com_75_anos_de_idade',
            'V110':'pessoas_com_76_anos_de_idade',
            'V111':'pessoas_com_77_anos_de_idade',
            'V112':'pessoas_com_78_anos_de_idade',
            'V113':'pessoas_com_79_anos_de_idade',
            'V114':'pessoas_com_80_anos_de_idade',
            'V115':'pessoas_com_81_anos_de_idade',
            'V116':'pessoas_com_82_anos_de_idade',
            'V117':'pessoas_com_83_anos_de_idade',
            'V118':'pessoas_com_84_anos_de_idade',
            'V119':'pessoas_com_85_anos_de_idade',
            'V120':'pessoas_com_86_anos_de_idade',
            'V121':'pessoas_com_87_anos_de_idade',
            'V122':'pessoas_com_88_anos_de_idade',
            'V123':'pessoas_com_89_anos_de_idade',
            'V124':'pessoas_com_90_anos_de_idade',
            'V125':'pessoas_com_91_anos_de_idade',
            'V126':'pessoas_com_92_anos_de_idade',
            'V127':'pessoas_com_93_anos_de_idade',
            'V128':'pessoas_com_94_anos_de_idade',
            'V129':'pessoas_com_95_anos_de_idade',
            'V130':'pessoas_com_96_anos_de_idade',
            'V131':'pessoas_com_97_anos_de_idade',
            'V132':'pessoas_com_98_anos_de_idade',
            'V133':'pessoas_com_99_anos_de_idade',
            'V134':'pessoas_com_100_anos_ou_mais_de_idade',
        },
        # ARQUIVO_PESSOA11_UF.XLS    
        {
            'V001':'homens_residentes_em_domicilios_particulares_e_domicilios_coletivos'
        },
        # ARQUIVO_PESSOA12_UF.XLS
        {
            'V001':'mulheres_residentes_em_domicilios_particulares_e_domicilios_coletivos'
        },    
    ]
    
    # 4. Lê cada um dos arquivos do censo de acordo com os parâmetros previamente definidos
    dados_censo = []
    for pattern, correspondence in zip(patterns, correspondences):
        read_data(pattern, correspondence, dados_censo)
        
    # 5. Reduce itera par a par nos itens da lista que foi preenchida por read_data. Depois, o merge é feito com lambda
    dados_censo = merge_censo(dados_censo)
    
    # 6. Resolve caso dos campos com 'X' para setores anonimizados
    dados_censo = handle_anom_data(dados_censo, correspondences)
    
    # 7. Salva para reiniciar sem esperar ler e dar merge em todos os arquivos, quando necessário
    dados_censo.to_csv("../data/dados-censo-concatenados-temp.csv", index=False)
    setores.to_file("../data/geo/setores-concatenados-temp.shp")
    
    return dados_censo, setores

Agora, podemos rodá-la.

In [ ]:
dados_censo, setores = run_censo_concat()

A função anterior salvou um arquivo `.csv` e um `.shp`. Vamos abrí-los. Assim, evitamos o tempo de carregamento anterior.

In [ ]:
%%time
# Lê novamente
dtype = {
            'Cod_setor':'str',                       
            'Cod_Grandes Regiões':'str',                            
            'Cod_UF':'str',                                        
            'Cod_meso':'str',                                       
            'Cod_micro':'str',                                      
            'Cod_RM':'str',                                         
            'Cod_municipio':'str',                                  
            'Cod_distrito':'str',                                   
            'Cod_subdistrito':'str',                                
            'Cod_bairro':'str',                                     
            'Situacao_setor':'str',
            'Tipo_setor':'str',                                  
        }
dados_censo = pd.read_csv("../data/dados-censo-concatenados-temp.csv", dtype=dtype)
setores = gpd.read_file("../data/geo/setores-concatenados-temp.shp")

### 7. Criar campos percentuais para os valores de cada setor censitário.

A variável `dados_censo` é um grande dataframe com informações úteis. Entretanto, elas estão mais granulares do que gostaríamos. Vamos definir funções para extrair dados mais consolidados.

Queremos:
- Total de eleitores jovens (16 - 24 anos)
- Total de pessoas alfabetizadas
- Total de eleitores alfabetizados
- Total de eleitores mulheres
- Total de eleitores homens
- Total de mulheres responsáveis por domicílio
- Total de negros, brancos, pardos, amarelos e indígenas

A função abaixo calcula esses percentuais. Ela recebe uma lista de campos para somar e o nome do campo que será retornado. Vamos usar ela em um loop para salvar tudo que precisamos.

In [ ]:
def get_aggregates(fields_to_sum, agg_field_name, df):
        total_of_group = df[fields_to_sum].sum(axis=1)
        df[agg_field_name] = total_of_group

In [ ]:
def run_aggregation(df):
    
    # Cópia para não alterar localmente
    temp_df = df.copy()

    # Define as variáveis necessárias para agregar
    pessoas_em_idade_eleitoral = [ 
            'pessoas_com_16_anos_de_idade',
            'pessoas_com_17_anos_de_idade',
            'pessoas_com_18_anos_de_idade',
            'pessoas_com_19_anos_de_idade',
            'pessoas_com_20_anos_de_idade',
            'pessoas_com_21_anos_de_idade',
            'pessoas_com_22_anos_de_idade',
            'pessoas_com_23_anos_de_idade',
            'pessoas_com_24_anos_de_idade',
            'pessoas_com_25_anos_de_idade',
            'pessoas_com_26_anos_de_idade',
            'pessoas_com_27_anos_de_idade',
            'pessoas_com_28_anos_de_idade',
            'pessoas_com_29_anos_de_idade',
            'pessoas_com_30_anos_de_idade',
            'pessoas_com_31_anos_de_idade',
            'pessoas_com_32_anos_de_idade',
            'pessoas_com_33_anos_de_idade',
            'pessoas_com_34_anos_de_idade',
            'pessoas_com_35_anos_de_idade',
            'pessoas_com_36_anos_de_idade',
            'pessoas_com_37_anos_de_idade',
            'pessoas_com_38_anos_de_idade',
            'pessoas_com_39_anos_de_idade',
            'pessoas_com_40_anos_de_idade',
            'pessoas_com_41_anos_de_idade',
            'pessoas_com_42_anos_de_idade',
            'pessoas_com_43_anos_de_idade',
            'pessoas_com_44_anos_de_idade',
            'pessoas_com_45_anos_de_idade',
            'pessoas_com_46_anos_de_idade',
            'pessoas_com_47_anos_de_idade',
            'pessoas_com_48_anos_de_idade',
            'pessoas_com_49_anos_de_idade',
            'pessoas_com_50_anos_de_idade',
            'pessoas_com_51_anos_de_idade',
            'pessoas_com_52_anos_de_idade',
            'pessoas_com_53_anos_de_idade',
            'pessoas_com_54_anos_de_idade',
            'pessoas_com_55_anos_de_idade',
            'pessoas_com_56_anos_de_idade',
            'pessoas_com_57_anos_de_idade',
            'pessoas_com_58_anos_de_idade',
            'pessoas_com_59_anos_de_idade',
            'pessoas_com_60_anos_de_idade',
            'pessoas_com_61_anos_de_idade',
            'pessoas_com_62_anos_de_idade',
            'pessoas_com_63_anos_de_idade',
            'pessoas_com_64_anos_de_idade',
            'pessoas_com_65_anos_de_idade',
            'pessoas_com_66_anos_de_idade',
            'pessoas_com_67_anos_de_idade',
            'pessoas_com_68_anos_de_idade',
            'pessoas_com_69_anos_de_idade',
            'pessoas_com_70_anos_de_idade',
            'pessoas_com_71_anos_de_idade',
            'pessoas_com_72_anos_de_idade',
            'pessoas_com_73_anos_de_idade',
            'pessoas_com_74_anos_de_idade',
            'pessoas_com_75_anos_de_idade',
            'pessoas_com_76_anos_de_idade',
            'pessoas_com_77_anos_de_idade',
            'pessoas_com_78_anos_de_idade',
            'pessoas_com_79_anos_de_idade',
            'pessoas_com_80_anos_de_idade',
            'pessoas_com_81_anos_de_idade',
            'pessoas_com_82_anos_de_idade',
            'pessoas_com_83_anos_de_idade',
            'pessoas_com_84_anos_de_idade',
            'pessoas_com_85_anos_de_idade',
            'pessoas_com_86_anos_de_idade',
            'pessoas_com_87_anos_de_idade',
            'pessoas_com_88_anos_de_idade',
            'pessoas_com_89_anos_de_idade',
            'pessoas_com_90_anos_de_idade',
            'pessoas_com_91_anos_de_idade',
            'pessoas_com_92_anos_de_idade',
            'pessoas_com_93_anos_de_idade',
            'pessoas_com_94_anos_de_idade',
            'pessoas_com_95_anos_de_idade',
            'pessoas_com_96_anos_de_idade',
            'pessoas_com_97_anos_de_idade',
            'pessoas_com_98_anos_de_idade',
            'pessoas_com_99_anos_de_idade',
            'pessoas_com_100_anos_ou_mais_de_idade' 
        ]

    eleitores_de_16_a_24_anos = [ 
                'pessoas_com_16_anos_de_idade',
                'pessoas_com_17_anos_de_idade',
                'pessoas_com_18_anos_de_idade',
                'pessoas_com_19_anos_de_idade',
                'pessoas_com_20_anos_de_idade',
                'pessoas_com_21_anos_de_idade',
                'pessoas_com_22_anos_de_idade',
                'pessoas_com_23_anos_de_idade',
                'pessoas_com_24_anos_de_idade',
        ]

    total_alfabetizados = [
                            'pessoas_alfabetizadas_com_5_ou_mais_anos_de_idade'
                          ]

    eleitores_alfabetizados = [
                                'pessoas_alfabetizadas_com_16_anos_de_idade',
                                'pessoas_alfabetizadas_com_17_anos_de_idade',
                                'pessoas_alfabetizadas_com_18_anos_de_idade',
                                'pessoas_alfabetizadas_com_19_anos_de_idade',
                                'pessoas_alfabetizadas_com_20_anos_de_idade',
                                'pessoas_alfabetizadas_com_21_anos_de_idade',
                                'pessoas_alfabetizadas_com_22_anos_de_idade',
                                'pessoas_alfabetizadas_com_23_anos_de_idade',
                                'pessoas_alfabetizadas_com_24_anos_de_idade',
                                'pessoas_alfabetizadas_com_25_anos_de_idade',
                                'pessoas_alfabetizadas_com_26_anos_de_idade',
                                'pessoas_alfabetizadas_com_27_anos_de_idade',
                                'pessoas_alfabetizadas_com_28_anos_de_idade',
                                'pessoas_alfabetizadas_com_29_anos_de_idade',
                                'pessoas_alfabetizadas_com_30_anos_de_idade',
                                'pessoas_alfabetizadas_com_31_anos_de_idade',
                                'pessoas_alfabetizadas_com_32_anos_de_idade',
                                'pessoas_alfabetizadas_com_33_anos_de_idade',
                                'pessoas_alfabetizadas_com_34_anos_de_idade',
                                'pessoas_alfabetizadas_com_35_anos_de_idade',
                                'pessoas_alfabetizadas_com_36_anos_de_idade',
                                'pessoas_alfabetizadas_com_37_anos_de_idade',
                                'pessoas_alfabetizadas_com_38_anos_de_idade',
                                'pessoas_alfabetizadas_com_39_anos_de_idade',
                                'pessoas_alfabetizadas_com_40_anos_de_idade',
                                'pessoas_alfabetizadas_com_41_anos_de_idade',
                                'pessoas_alfabetizadas_com_42_anos_de_idade',
                                'pessoas_alfabetizadas_com_43_anos_de_idade',
                                'pessoas_alfabetizadas_com_44_anos_de_idade',
                                'pessoas_alfabetizadas_com_45_anos_de_idade',
                                'pessoas_alfabetizadas_com_46_anos_de_idade',
                                'pessoas_alfabetizadas_com_47_anos_de_idade',
                                'pessoas_alfabetizadas_com_48_anos_de_idade',
                                'pessoas_alfabetizadas_com_49_anos_de_idade',
                                'pessoas_alfabetizadas_com_50_anos_de_idade',
                                'pessoas_alfabetizadas_com_51_anos_de_idade',
                                'pessoas_alfabetizadas_com_52_anos_de_idade',
                                'pessoas_alfabetizadas_com_53_anos_de_idade',
                                'pessoas_alfabetizadas_com_54_anos_de_idade',
                                'pessoas_alfabetizadas_com_55_anos_de_idade',
                                'pessoas_alfabetizadas_com_56_anos_de_idade',
                                'pessoas_alfabetizadas_com_57_anos_de_idade',
                                'pessoas_alfabetizadas_com_58_anos_de_idade',
                                'pessoas_alfabetizadas_com_59_anos_de_idade',
                                'pessoas_alfabetizadas_com_60_anos_de_idade',
                                'pessoas_alfabetizadas_com_61_anos_de_idade',
                                'pessoas_alfabetizadas_com_62_anos_de_idade',
                                'pessoas_alfabetizadas_com_63_anos_de_idade',
                                'pessoas_alfabetizadas_com_64_anos_de_idade',
                                'pessoas_alfabetizadas_com_65_anos_de_idade',
                                'pessoas_alfabetizadas_com_66_anos_de_idade',
                                'pessoas_alfabetizadas_com_67_anos_de_idade',
                                'pessoas_alfabetizadas_com_68_anos_de_idade',
                                'pessoas_alfabetizadas_com_69_anos_de_idade',
                                'pessoas_alfabetizadas_com_70_anos_de_idade',
                                'pessoas_alfabetizadas_com_71_anos_de_idade',
                                'pessoas_alfabetizadas_com_72_anos_de_idade',
                                'pessoas_alfabetizadas_com_73_anos_de_idade',
                                'pessoas_alfabetizadas_com_74_anos_de_idade',
                                'pessoas_alfabetizadas_com_75_anos_de_idade',
                                'pessoas_alfabetizadas_com_76_anos_de_idade',
                                'pessoas_alfabetizadas_com_77_anos_de_idade',
                                'pessoas_alfabetizadas_com_78_anos_de_idade',
                                'pessoas_alfabetizadas_com_79_anos_de_idade',
                                'pessoas_alfabetizadas_com_80_anos_ou_mais_de_idade',
                            ]

    total_homens = ['homens_residentes_em_domicilios_particulares_e_domicilios_coletivos']

    total_mulheres = ['mulheres_residentes_em_domicilios_particulares_e_domicilios_coletivos']

    total_mulheres_responsaveis_por_domicilio = [ 
                                                 'domicílios_particulares_permanentes_com_mulher_responsável_e_mais_1_morador',
                                                 'domicílios_particulares_permanentes_com_mulher_responsável_e_mais_2_moradores',
                                                 'domicílios_particulares_permanentes_com_mulher_responsável_e_mais_3_moradores',
                                                 'domicílios_particulares_permanentes_com_mulher_responsável_e_mais_4_moradores',
                                                 'domicílios_particulares_permanentes_com_mulher_responsável_e_mais_5_moradores',
                                                 'domicílios_particulares_permanentes_com_mulher_responsável_e_mais_6_ou_mais_moradores',
                                                 'domicílios_particulares_permanentes_com_mulher_responsáVel_e_sem_outro_morador'
                                                ]

    total_pretos = ['pessoas_residentes_e_cor_ou_raça_preta']

    total_pardos = ['pessoas_residentes_e_cor_ou_raça_parda']

    total_pretos_e_pardos = ['pessoas_residentes_e_cor_ou_raça_preta', 'pessoas_residentes_e_cor_ou_raça_parda']

    total_brancos = ['pessoas_residentes_e_cor_ou_raça_branca']

    total_indigenas = ['pessoas_residentes_e_cor_ou_raça_indígenas']

    total_amarelos = ['pessoas_residentes_e_cor_ou_raça_amarela']
    
    # Lista os campos que precisam ser somados - é uma lista das listas criadas acima
    fields_to_sum = [ 
        pessoas_em_idade_eleitoral, eleitores_de_16_a_24_anos, total_alfabetizados, eleitores_alfabetizados, total_homens, total_mulheres, 
        total_mulheres_responsaveis_por_domicilio, total_pretos, total_pardos, total_pretos_e_pardos,
        total_brancos, total_indigenas, total_amarelos, 
    ]
    
    # Define as labels dos resultados
    result_labels = [ 
     "pessoas_em_idade_eleitoral", "eleitores_de_16_a_24_anos", "total_alfabetizados", "eleitores_alfabetizados", "total_homens", "total_mulheres", 
     "total_mulheres_responsaveis_por_domicilio", "total_pretos", "total_pardos", "total_pretos_e_pardos",
     "total_brancos", "total_indigenas", "total_amarelos", 
    ]
    
    # Roda a função, modificando o df inplace
    for fields, result_label in zip(fields_to_sum, result_labels):
        get_aggregates(fields, result_label, temp_df)
        
    # Filtra, novamente inplace, para manter apenas as colunas relevantes
    to_keep = [
        'Cod_setor', 'Cod_Grandes Regiões', 'Nome_Grande_Regiao', 'Cod_UF',
        'Nome_da_UF ', 'Cod_meso', 'Nome_da_meso', 'Cod_micro', 'Nome_da_micro',
        'Cod_RM', 'Nome_da_RM', 'Cod_municipio', 'Nome_do_municipio',
        'Cod_distrito', 'Nome_do_distrito', 'Cod_subdistrito',
        'Nome_do_subdistrito', 'Cod_bairro', 'Nome_do_bairro', 'Situacao_setor',
        'Tipo_setor', 'pessoas_residentes', 
        'media_renda_nom_mensal_pessoas_com_10_anos_ou_mais_com_ou_sem_rendimento',
    ]
    ages = [
            'pessoas_com_menos_de_1_ano_de_idade',
            'pessoas_de_1_ano_de_idade',
            'pessoas_com_2_anos_de_idade',
            'pessoas_com_3_anos_de_idade',
            'pessoas_com_4_anos_de_idade',
            'pessoas_com_5_anos_de_idade',
            'pessoas_com_6_anos_de_idade',
            'pessoas_com_7_anos_de_idade',
            'pessoas_com_8_anos_de_idade',
            'pessoas_com_9_anos_de_idade',
            'pessoas_com_10_anos_de_idade',
            'pessoas_com_11_anos_de_idade',
            'pessoas_com_12_anos_de_idade',
            'pessoas_com_13_anos_de_idade',
            'pessoas_com_14_anos_de_idade',
            'pessoas_com_15_anos_de_idade',
            'pessoas_com_16_anos_de_idade',
            'pessoas_com_17_anos_de_idade',
            'pessoas_com_18_anos_de_idade',
            'pessoas_com_19_anos_de_idade',
            'pessoas_com_20_anos_de_idade',
            'pessoas_com_21_anos_de_idade',
            'pessoas_com_22_anos_de_idade',
            'pessoas_com_23_anos_de_idade',
            'pessoas_com_24_anos_de_idade',
            'pessoas_com_25_anos_de_idade',
            'pessoas_com_26_anos_de_idade',
            'pessoas_com_27_anos_de_idade',
            'pessoas_com_28_anos_de_idade',
            'pessoas_com_29_anos_de_idade',
            'pessoas_com_30_anos_de_idade',
            'pessoas_com_31_anos_de_idade',
            'pessoas_com_32_anos_de_idade',
            'pessoas_com_33_anos_de_idade',
            'pessoas_com_34_anos_de_idade',
            'pessoas_com_35_anos_de_idade',
            'pessoas_com_36_anos_de_idade',
            'pessoas_com_37_anos_de_idade',
            'pessoas_com_38_anos_de_idade',
            'pessoas_com_39_anos_de_idade',
            'pessoas_com_40_anos_de_idade',
            'pessoas_com_41_anos_de_idade',
            'pessoas_com_42_anos_de_idade',
            'pessoas_com_43_anos_de_idade',
            'pessoas_com_44_anos_de_idade',
            'pessoas_com_45_anos_de_idade',
            'pessoas_com_46_anos_de_idade',
            'pessoas_com_47_anos_de_idade',
            'pessoas_com_48_anos_de_idade',
            'pessoas_com_49_anos_de_idade',
            'pessoas_com_50_anos_de_idade',
            'pessoas_com_51_anos_de_idade',
            'pessoas_com_52_anos_de_idade',
            'pessoas_com_53_anos_de_idade',
            'pessoas_com_54_anos_de_idade',
            'pessoas_com_55_anos_de_idade',
            'pessoas_com_56_anos_de_idade',
            'pessoas_com_57_anos_de_idade',
            'pessoas_com_58_anos_de_idade',
            'pessoas_com_59_anos_de_idade',
            'pessoas_com_60_anos_de_idade',
            'pessoas_com_61_anos_de_idade',
            'pessoas_com_62_anos_de_idade',
            'pessoas_com_63_anos_de_idade',
            'pessoas_com_64_anos_de_idade',
            'pessoas_com_65_anos_de_idade',
            'pessoas_com_66_anos_de_idade',
            'pessoas_com_67_anos_de_idade',
            'pessoas_com_68_anos_de_idade',
            'pessoas_com_69_anos_de_idade',
            'pessoas_com_70_anos_de_idade',
            'pessoas_com_71_anos_de_idade',
            'pessoas_com_72_anos_de_idade',
            'pessoas_com_73_anos_de_idade',
            'pessoas_com_74_anos_de_idade',
            'pessoas_com_75_anos_de_idade',
            'pessoas_com_76_anos_de_idade',
            'pessoas_com_77_anos_de_idade',
            'pessoas_com_78_anos_de_idade',
            'pessoas_com_79_anos_de_idade',
            'pessoas_com_80_anos_de_idade',
            'pessoas_com_81_anos_de_idade',
            'pessoas_com_82_anos_de_idade',
            'pessoas_com_83_anos_de_idade',
            'pessoas_com_84_anos_de_idade',
            'pessoas_com_85_anos_de_idade',
            'pessoas_com_86_anos_de_idade',
            'pessoas_com_87_anos_de_idade',
            'pessoas_com_88_anos_de_idade',
            'pessoas_com_89_anos_de_idade',
            'pessoas_com_90_anos_de_idade',
            'pessoas_com_91_anos_de_idade',
            'pessoas_com_92_anos_de_idade',
            'pessoas_com_93_anos_de_idade',
            'pessoas_com_94_anos_de_idade',
            'pessoas_com_95_anos_de_idade',
            'pessoas_com_96_anos_de_idade',
            'pessoas_com_97_anos_de_idade',
            'pessoas_com_98_anos_de_idade',
            'pessoas_com_99_anos_de_idade',
            'pessoas_com_100_anos_ou_mais_de_idade',
        ]
    to_keep.extend(result_labels)
    to_keep.extend(ages)    
    temp_df = temp_df[to_keep]
    
    return temp_df

In [ ]:
dados_censo = run_aggregation(dados_censo)

Vamos unir esses dados aos shapefiles com base nos campos `CD_GEOCODI` e `Cod_setor`. Nem todos os setores do shapefile vão ter dados associados: sobram vazios demográficos. Eles desaparecem no merge.

In [ ]:
setores = setores.merge(dados_censo, left_on='CD_GEOCODI', right_on='Cod_setor', how='inner', suffixes=('','__y'))
setores = setores[[col for col in setores.columns if '__y' not in col]]

Precisamos, também, colocar ambos no mesmo crs.

In [ ]:
setores = setores.to_crs(voronois.crs)

Agora podemos definir uma rotina para aproximar a população de cada voronoi.

### 7. Estimar população dos voronois
Agora já temos todos os elementos necessários para estimar as características da população que vive em cada voronoi. As funções abaixo, posteriormente encapsuladas em uma único, fazem esse cálculo passo a passo.

In [ ]:
voronois = voronois.reset_index()
voronois = voronois.drop(['index'], axis=1)

Primeiro, cada voronoi deve ser tratado como um objeto geográfico do GeoPandas, para assim permitir o cálculo da interseção.

In [ ]:
def make_geodf(row):
    # A função pega um GeoSeries e retorna um GeoDataFrame para possibilitar
    # operações geográficas no GeoPandas
    geo_data = row.geometry
    geo_data = gpd.GeoSeries(geo_data)
    geo_data = gpd.GeoDataFrame(geo_data)
    geo_data.columns = ['geometry']
    geo_data.crs = {'init': 'epsg:4326'}
    return geo_data

Depois, para cada um deses novos GeoDataFrames, selecionamos os setores censitários dentro dele.

In [ ]:
def seleciona_setores(gdf_voronoi, gdf_setores, city_id, whole_country=False):
    # Seleciona os setores censitários que estão dentro do voronoi.
    # O parâmetro whole_country deve ser true para buscar interseções em todo o país,
    # não apenas no contorno do município.
    
    temp = gdf_setores.copy()
    
    if whole_country is False:
        temp = gdf_setores[gdf_setores.CD_GEOCODM==city_id]
        
    setores_dentro = gpd.sjoin(temp, gdf_voronoi, how='inner', op='intersects')
    return setores_dentro

In [ ]:
def calcula_intersecao(gdf_voronoi, row):
    # Calcula a intereseção do setor com o voronoi
    
    gdf_voronoi.geometry = gdf_voronoi.geometry.buffer(0)
    intersection = gdf_voronoi.intersection(row.geometry.buffer(0))
    intersection_area = float(intersection.area)
    intersection_percentage = float(intersection_area / row.geometry.area)
    
    return intersection_percentage

In [ ]:
def calcula_valores(voronoi_totals, intersection_percentage, row):
    # Usa o valor retornado por calcula_intersecao para adicionar valores
    # para cada uma das colunas de dados do voronoi.
    # voronoi_totals é modificado inplace, sem necessidade de retornar um valor
    
    for k,v in voronoi_totals.items(): # voronoi_totals é um elemento criado a partir da variável externa 'keys')
        
        # Essa condição é para realizar o cálculo de QUANTO DINHEIRO entra, não um percentual da média
        # de acordo com a interseção. Trata-se da única variável em que o valor medido não é uma contagem
        # de pessoas. Note que, apesar de chamar 'media', nesse momento do fluxo, o valor representa
        # uma soma absoluta.
        if k == 'media_renda_nom_mensal_pessoas_com_10_anos_ou_mais_com_ou_sem_rendimento':
            # IMPLEMENTAR
            people = intersection_percentage * row['pessoas_residentes']
            to_add = round(people * row[k], 2)
        else:
            to_add = round(intersection_percentage * row[k], 2) # setor[k] é um campo de dados da linha
        
        voronoi_totals[k] = voronoi_totals[k] + to_add


In [ ]:
# Rodar via df.apply(f, axis=1)
def preenche_dados_voronois(row, keys, gdf_setores, whole_country=False):
    
    # Essa função usa o percentual de interseção e os dados do censo para
    # descobrir quantas pessoas de cada característica moram em
    
    city_id = row.COD_IBGE
    voronoi = make_geodf(row)
    voronoi_totals = {key: 0 for key in keys}
    setores_dentro = seleciona_setores(voronoi, gdf_setores, city_id, whole_country)
    
    if setores_dentro.shape[0] == 0:
        return pd.Series(voronoi_totals)

    for index, row_ in setores_dentro.iterrows():
        intersection_percentage = calcula_intersecao(voronoi, row_)
        calcula_valores(voronoi_totals, intersection_percentage, row_)
        
    # Calcula uma média de renda
    try:
        voronoi_totals['media_renda_nom_mensal_pessoas_com_10_anos_ou_mais_com_ou_sem_rendimento'] = voronoi_totals['media_renda_nom_mensal_pessoas_com_10_anos_ou_mais_com_ou_sem_rendimento'] / voronoi_totals['pessoas_residentes']  
    except ZeroDivisionError:
        voronoi_totals['media_renda_nom_mensal_pessoas_com_10_anos_ou_mais_com_ou_sem_rendimento'] = np.nan
        
    return pd.Series(voronoi_totals)

In [ ]:
def calcular_percentuais(row, exceptions):
    columns = ['eleitores_alfabetizados', 'eleitores_de_16_a_24_anos',
   'pessoas_em_idade_eleitoral', 'pessoas_residentes',
   'total_alfabetizados', 'total_amarelos', 'total_brancos',
   'total_homens', 'total_indigenas', 'total_mulheres',
   'total_mulheres_responsaveis_por_domicilio', 'total_pardos',
   'total_pretos', 'total_pretos_e_pardos']
    
    percent_values = { 'pp_' + key:None for key in columns }
    
    for column in columns:
        #print(column)
        if column in ['eleitores_alfabetizados', 'eleitores_de_16_a_24_anos']:
            col_to_divide = 'pessoas_em_idade_eleitoral'            
        else:
            col_to_divide = 'pessoas_residentes'

        try:
            #print(row[column], row[col_to_divide])
            percent_values['pp_' + column] = row[column] / row[col_to_divide]
        except ZeroDivisionError as e:
            #print("ZeroDivisionError at index", row.name)
            #print("error")
            exceptions.append(row.name)
            percent_values['pp_' + column] = 0

    return pd.Series(percent_values)

In [ ]:
def estimate_voronoi_census_data(setores_arg, voronois_arg):
    
    # Cópias locais
    gdf_setores = setores_arg.copy().reset_index()
    if 'media_renda_nom_mensal_pessoas_com_10_anos_ou_mais_com_ou_sem_rendimento' in gdf_setores.columns:
        print("column is here")
    
    gdf_voronois = voronois_arg.copy().reset_index()
    print("Copy made")
    
    # Coloca voronois e setores no mesmo crs
    gdf_setores = gdf_setores.to_crs(gdf_voronois.crs)
    if gdf_setores.crs == gdf_voronois.crs:
        pass
    else:
        print("Wrong crs")
        return None
    
    print("Crs converted")
    
    # Cria as chaves dos campos calculados
    ages = [
            'pessoas_com_menos_de_1_ano_de_idade',
            'pessoas_de_1_ano_de_idade',
            'pessoas_com_2_anos_de_idade',
            'pessoas_com_3_anos_de_idade',
            'pessoas_com_4_anos_de_idade',
            'pessoas_com_5_anos_de_idade',
            'pessoas_com_6_anos_de_idade',
            'pessoas_com_7_anos_de_idade',
            'pessoas_com_8_anos_de_idade',
            'pessoas_com_9_anos_de_idade',
            'pessoas_com_10_anos_de_idade',
            'pessoas_com_11_anos_de_idade',
            'pessoas_com_12_anos_de_idade',
            'pessoas_com_13_anos_de_idade',
            'pessoas_com_14_anos_de_idade',
            'pessoas_com_15_anos_de_idade',
            'pessoas_com_16_anos_de_idade',
            'pessoas_com_17_anos_de_idade',
            'pessoas_com_18_anos_de_idade',
            'pessoas_com_19_anos_de_idade',
            'pessoas_com_20_anos_de_idade',
            'pessoas_com_21_anos_de_idade',
            'pessoas_com_22_anos_de_idade',
            'pessoas_com_23_anos_de_idade',
            'pessoas_com_24_anos_de_idade',
            'pessoas_com_25_anos_de_idade',
            'pessoas_com_26_anos_de_idade',
            'pessoas_com_27_anos_de_idade',
            'pessoas_com_28_anos_de_idade',
            'pessoas_com_29_anos_de_idade',
            'pessoas_com_30_anos_de_idade',
            'pessoas_com_31_anos_de_idade',
            'pessoas_com_32_anos_de_idade',
            'pessoas_com_33_anos_de_idade',
            'pessoas_com_34_anos_de_idade',
            'pessoas_com_35_anos_de_idade',
            'pessoas_com_36_anos_de_idade',
            'pessoas_com_37_anos_de_idade',
            'pessoas_com_38_anos_de_idade',
            'pessoas_com_39_anos_de_idade',
            'pessoas_com_40_anos_de_idade',
            'pessoas_com_41_anos_de_idade',
            'pessoas_com_42_anos_de_idade',
            'pessoas_com_43_anos_de_idade',
            'pessoas_com_44_anos_de_idade',
            'pessoas_com_45_anos_de_idade',
            'pessoas_com_46_anos_de_idade',
            'pessoas_com_47_anos_de_idade',
            'pessoas_com_48_anos_de_idade',
            'pessoas_com_49_anos_de_idade',
            'pessoas_com_50_anos_de_idade',
            'pessoas_com_51_anos_de_idade',
            'pessoas_com_52_anos_de_idade',
            'pessoas_com_53_anos_de_idade',
            'pessoas_com_54_anos_de_idade',
            'pessoas_com_55_anos_de_idade',
            'pessoas_com_56_anos_de_idade',
            'pessoas_com_57_anos_de_idade',
            'pessoas_com_58_anos_de_idade',
            'pessoas_com_59_anos_de_idade',
            'pessoas_com_60_anos_de_idade',
            'pessoas_com_61_anos_de_idade',
            'pessoas_com_62_anos_de_idade',
            'pessoas_com_63_anos_de_idade',
            'pessoas_com_64_anos_de_idade',
            'pessoas_com_65_anos_de_idade',
            'pessoas_com_66_anos_de_idade',
            'pessoas_com_67_anos_de_idade',
            'pessoas_com_68_anos_de_idade',
            'pessoas_com_69_anos_de_idade',
            'pessoas_com_70_anos_de_idade',
            'pessoas_com_71_anos_de_idade',
            'pessoas_com_72_anos_de_idade',
            'pessoas_com_73_anos_de_idade',
            'pessoas_com_74_anos_de_idade',
            'pessoas_com_75_anos_de_idade',
            'pessoas_com_76_anos_de_idade',
            'pessoas_com_77_anos_de_idade',
            'pessoas_com_78_anos_de_idade',
            'pessoas_com_79_anos_de_idade',
            'pessoas_com_80_anos_de_idade',
            'pessoas_com_81_anos_de_idade',
            'pessoas_com_82_anos_de_idade',
            'pessoas_com_83_anos_de_idade',
            'pessoas_com_84_anos_de_idade',
            'pessoas_com_85_anos_de_idade',
            'pessoas_com_86_anos_de_idade',
            'pessoas_com_87_anos_de_idade',
            'pessoas_com_88_anos_de_idade',
            'pessoas_com_89_anos_de_idade',
            'pessoas_com_90_anos_de_idade',
            'pessoas_com_91_anos_de_idade',
            'pessoas_com_92_anos_de_idade',
            'pessoas_com_93_anos_de_idade',
            'pessoas_com_94_anos_de_idade',
            'pessoas_com_95_anos_de_idade',
            'pessoas_com_96_anos_de_idade',
            'pessoas_com_97_anos_de_idade',
            'pessoas_com_98_anos_de_idade',
            'pessoas_com_99_anos_de_idade',
            'pessoas_com_100_anos_ou_mais_de_idade',
        ]
    keys = [ 
     "pessoas_em_idade_eleitoral", "eleitores_de_16_a_24_anos", "total_alfabetizados", "eleitores_alfabetizados", "total_homens", "total_mulheres", 
     "total_mulheres_responsaveis_por_domicilio", "total_pretos", "total_pardos", "total_pretos_e_pardos",
     "total_brancos", "total_indigenas", "total_amarelos", "pessoas_residentes", 
     'media_renda_nom_mensal_pessoas_com_10_anos_ou_mais_com_ou_sem_rendimento', # Essa última se tornará uma soma da renda das pessoas que entram no voronoi, assumindo que todas ganham a média
    ]
    keys.extend(ages)
    
    # Roda a função que preenche os dados de todas as cidades que existiam no censo 2010
    function_return = gdf_voronois.progress_apply(preenche_dados_voronois, axis=1, args=(keys, gdf_setores, False))
    gdf_voronois = pd.concat([gdf_voronois, function_return], axis=1).reset_index(drop=True)    
    print("First batch of data calculated")
    
    # Cinco cidades que foram criadas após a realização do Censo não puderam ter os valores preenchidos. 
    # Agora, realizamos o processo de novo, mas com um sjoin que envolve todo o país. 
    # Isso é possível porque os setores censitários cobrem todo o território nacional. 
    # Assim, vamos capturar os dados referentes aos distritos dos quais essas novas cidades se desmembraram.
    function_return = (gdf_voronois[gdf_voronois.pessoas_residentes==0]).progress_apply(preenche_dados_voronois, axis=1, args=(keys, gdf_setores, True))
    print("Second batch of data calculated")
    
    # Substitui essasa novas cidades diretamente no índice
    gdf_voronois.loc[function_return.index, function_return.columns] = function_return # verificar se erro não pode estar sendo gerado aqui
    
    # Cria campos que recebem valores percentuais
    exceptions = []
    function_return = gdf_voronois.progress_apply(calcular_percentuais, axis=1, args=(exceptions,))
    gdf_voronois = pd.concat([gdf_voronois, function_return], axis=1).reset_index(drop=True)
    print("Percentages calculated")
    
    # Renomeia colunas
    columns = {
       'index':'INDEX',
       'COD_TSE':'COD_TSE',
       'COD_IBGE':'COD_IBGE',
       'NM_MUNICIP':'NM_MUNIC',
       'UF':'UF',
       'ZONA':'ZONA',
       'fetched_ad':'FET_AD',
       'id_unico':'ID_UNICO',
       'lat':'LAT',
       'geom_str':'GEOM_STR',
       'loc_unico':'LOC_UNICO',
       'lon':'LON',
       'geometry':'geometry',
       'eleitores_alfabetizados':'ELEI_ALF',
       'eleitores_de_16_a_24_anos':'ELEI_16_24',
       'pessoas_com_100_anos_ou_mais_de_idade':'P100_MAIS',
       'pessoas_com_10_anos_de_idade':'P10',
       'pessoas_com_11_anos_de_idade':'P11',
       'pessoas_com_12_anos_de_idade':'P12',
       'pessoas_com_13_anos_de_idade':'P13',
       'pessoas_com_14_anos_de_idade':'P14',
       'pessoas_com_15_anos_de_idade':'P15',
       'pessoas_com_16_anos_de_idade':'P16',
       'pessoas_com_17_anos_de_idade':'P17',
       'pessoas_com_18_anos_de_idade':'P18',
       'pessoas_com_19_anos_de_idade':'P19',
       'pessoas_com_20_anos_de_idade':'P20',
       'pessoas_com_21_anos_de_idade':'P21',
       'pessoas_com_22_anos_de_idade':'P22',
       'pessoas_com_23_anos_de_idade':'P23',
       'pessoas_com_24_anos_de_idade':'P24',
       'pessoas_com_25_anos_de_idade':'P25',
       'pessoas_com_26_anos_de_idade':'P26',
       'pessoas_com_27_anos_de_idade':'P27',
       'pessoas_com_28_anos_de_idade':'P28',
       'pessoas_com_29_anos_de_idade':'P29',
       'pessoas_com_2_anos_de_idade':'P2',
       'pessoas_com_30_anos_de_idade':'P30',
       'pessoas_com_31_anos_de_idade':'P31',
       'pessoas_com_32_anos_de_idade':'P32',
       'pessoas_com_33_anos_de_idade':'P33',
       'pessoas_com_34_anos_de_idade':'P34',
       'pessoas_com_35_anos_de_idade':'P35',
       'pessoas_com_36_anos_de_idade':'P36',
       'pessoas_com_37_anos_de_idade':'P37',
       'pessoas_com_38_anos_de_idade':'P38',
       'pessoas_com_39_anos_de_idade':'P39',
       'pessoas_com_3_anos_de_idade':'P3',
       'pessoas_com_40_anos_de_idade':'P40',
       'pessoas_com_41_anos_de_idade':'P41',
       'pessoas_com_42_anos_de_idade':'P42',
       'pessoas_com_43_anos_de_idade':'P43',
       'pessoas_com_44_anos_de_idade':'P44',
       'pessoas_com_45_anos_de_idade':'P45',
       'pessoas_com_46_anos_de_idade':'P46',
       'pessoas_com_47_anos_de_idade':'P47',
       'pessoas_com_48_anos_de_idade':'P48',
       'pessoas_com_49_anos_de_idade':'P49',
       'pessoas_com_4_anos_de_idade':'P4',
       'pessoas_com_50_anos_de_idade':'P50',
       'pessoas_com_51_anos_de_idade':'P51',
       'pessoas_com_52_anos_de_idade':'P52',
       'pessoas_com_53_anos_de_idade':'P53',
       'pessoas_com_54_anos_de_idade':'P54',
       'pessoas_com_55_anos_de_idade':'P55',
       'pessoas_com_56_anos_de_idade':'P56',
       'pessoas_com_57_anos_de_idade':'P57',
       'pessoas_com_58_anos_de_idade':'P58',
       'pessoas_com_59_anos_de_idade':'P59',
       'pessoas_com_5_anos_de_idade':'P5',
       'pessoas_com_60_anos_de_idade':'P60',
       'pessoas_com_61_anos_de_idade':'P61',
       'pessoas_com_62_anos_de_idade':'P62',
       'pessoas_com_63_anos_de_idade':'P63',
       'pessoas_com_64_anos_de_idade':'P64',
       'pessoas_com_65_anos_de_idade':'P65',
       'pessoas_com_66_anos_de_idade':'P66',
       'pessoas_com_67_anos_de_idade':'P67',
       'pessoas_com_68_anos_de_idade':'P68',
       'pessoas_com_69_anos_de_idade':'P69',
       'pessoas_com_6_anos_de_idade':'P6',
       'pessoas_com_70_anos_de_idade':'P70',
       'pessoas_com_71_anos_de_idade':'P71',
       'pessoas_com_72_anos_de_idade':'P72',
       'pessoas_com_73_anos_de_idade':'P73',
       'pessoas_com_74_anos_de_idade':'P74',
       'pessoas_com_75_anos_de_idade':'P75',
       'pessoas_com_76_anos_de_idade':'P76',
       'pessoas_com_77_anos_de_idade':'P77',
       'pessoas_com_78_anos_de_idade':'P78',
       'pessoas_com_79_anos_de_idade':'P79',
       'pessoas_com_7_anos_de_idade':'P7',
       'pessoas_com_80_anos_de_idade':'P80',
       'pessoas_com_81_anos_de_idade':'P81',
       'pessoas_com_82_anos_de_idade':'P82',
       'pessoas_com_83_anos_de_idade':'P83',
       'pessoas_com_84_anos_de_idade':'P84',
       'pessoas_com_85_anos_de_idade':'P85',
       'pessoas_com_86_anos_de_idade':'P86',
       'pessoas_com_87_anos_de_idade':'P87',
       'pessoas_com_88_anos_de_idade':'P88',
       'pessoas_com_89_anos_de_idade':'P89',
       'pessoas_com_8_anos_de_idade':'P8',
       'pessoas_com_90_anos_de_idade':'P90',
       'pessoas_com_91_anos_de_idade':'P91',
       'pessoas_com_92_anos_de_idade':'P92',
       'pessoas_com_93_anos_de_idade':'P93',
       'pessoas_com_94_anos_de_idade':'P94',
       'pessoas_com_95_anos_de_idade':'P95',
       'pessoas_com_96_anos_de_idade':'P96',
       'pessoas_com_97_anos_de_idade':'P97',
       'pessoas_com_98_anos_de_idade':'P98',
       'pessoas_com_99_anos_de_idade':'P99',
       'pessoas_com_9_anos_de_idade':'P9',
       'pessoas_com_menos_de_1_ano_de_idade':'P1_MENOS',
       'pessoas_de_1_ano_de_idade':'P1',
       'pessoas_em_idade_eleitoral':'P_ELEI',
       'pessoas_residentes':'P_RESID',
       'media_renda_nom_mensal_pessoas_com_10_anos_ou_mais_com_ou_sem_rendimento':'RENDA_MEDI',
       'total_alfabetizados':'TOTAL_ALF',
       'total_amarelos':'TT_AMARELO',
       'total_brancos':'TT_BRANCO',
       'total_homens':'TT_HOMEM',
       'total_indigenas':'TT_INDIO',
       'total_mulheres':'TT_MULH',
       'total_mulheres_responsaveis_por_domicilio':'TT_MULH_DO',
       'total_pardos':'TT_PARDO',
       'total_pretos':'TT_PRETO',
       'total_pretos_e_pardos':'TT_PRE_PAR',
       'pp_eleitores_alfabetizados':'PP_ELEIALF',
       'pp_eleitores_de_16_a_24_anos':'PP_ELE1624',
       'pp_pessoas_em_idade_eleitoral':'PP_IDD_ELE',
       'pp_pessoas_residentes':'PP_RESIDE',
       'pp_total_alfabetizados':'PP_TT_ALF',
       'pp_total_amarelos':'PP_TT_AMAR',
       'pp_total_brancos':'PP_TT_BRAN',
       'pp_total_homens':'PP_TT_HMM',
       'pp_total_indigenas':'PP_TT_INDI',
       'pp_total_mulheres':'PP_TT_MULH',
       'pp_total_mulheres_responsaveis_por_domicilio':'PP_MULH_DO',
       'pp_total_pardos':'PP_TT_PARD',
       'pp_total_pretos':'PP_TT_PRET',
       'pp_total_pretos_e_pardos':'PP_T_PREPA',
     }
    
    gdf_voronois = gdf_voronois.rename(columns=columns)
    to_keep = [col for col in gdf_voronois.columns if col in columns.values()]
    party_codes = [col for col in gdf_voronois.columns if col.isdigit()]
    to_keep.extend(party_codes)
    to_keep.extend([party_code + "_PP" for party_code in party_codes])
    gdf_voronois = gdf_voronois[to_keep]
    
    # Agora esse pedaço de código vai me fazer parecer um imbecil.
    # O geopandas se recusa a salvar o shapefile com todas as colunas de dados.
    # Entretanto, se eu salvar os polígonos em um shapefile, os dados em um csv
    # e depois lê-los do disco e fazer um merge em campo comum, eu consigo salvar tudo
    # em um mesmo shapefile. Vá entender!
    try:
        fp_a = "../data/geo/voronois-finalizados/polygons-" + str(datetime.now())[:19] + ".shp"
        fp_b = "../data/geo/voronois-finalizados/data-" + str(datetime.now())[:19] + ".csv"
        fp_c = "../data/geo/voronois-finalizados/polygons-with-data-" + str(datetime.now())[:19] + ".shp"
        
        gdf_voronois[['geometry','ID_UNICO']].to_file(fp_a)  
        gdf_voronois[[col for col in gdf_voronois.columns if col != 'geometry']].to_csv(fp_b, index=False)
        
        print("Files saved")
        
        # Re-lê os dados, faz merge e salva em um arquivo só
        a = gpd.read_file(fp_a)
        b = pd.read_csv(fp_b, dtype={"ID_UNICO":"str"})
        c = a.merge(b, on='ID_UNICO')        
        c.to_file(fp_c)
        
    except Exception as e:
        print("Exception happened when saving")
        print(e)
    
    return gdf_voronois, exceptions


In [ ]:
voros_, exceptions = estimate_voronoi_census_data(setores, voronois)